In [1]:
import pandas as pd
import openpyxl as ox
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import os
os.getcwd()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

This is a modification on Alidu's Code. I focus on cleaning the text description to tell whether a permit is a common area heat pump or multiple unit heat pump


In [ ]:
data_apt_time = pd.read_excel('01Data/Seattle_HP_Aptmt Units_Timeseries.xlsx')
data_apt_ht= pd.read_excel('./01Data/Seattle_HP_unit_apartment only_Unique address.xlsx')


In [3]:
data_apt_time.describe(include='object')
## check the first few rows
data_apt_time.head()

,Number,Quarter,Status,Address,BuildingName,GreenRating,LEEDtype,version,Submarket,City,Class,TrueOwner,Style,RentType,AffordableType,MarketSegment,Neighborhood,addr_clean,has heat pump
count,47905,47905,47905,47905,46519,3231,2204,2204,47905,47905,47905,46810,47622,47905,12790,47833,47905,47905,47905
unique,945,96,1,943,917,10,3,5,71,1,2,549,5,3,6,3,49,943,2
top,MBSEA197,2023 Q4 QTD,Existing,1307 N 43rd St,Madison Court,Silver,LEED BD+C: New Construction,v2008,Capitol Hill,Seattle,B,Equity Residential,Mid-Rise,Market,Rent Restricted,All,CAPITOL HILL,1307 N 43RD ST,No
freq,96,945,47905,96,192,1045,1078,1042,5394,47905,35343,1361,34434,35138,7451,45655,5478,96,30433


,Number,Quarter,Status,AskingRentPerUnit,AskingRentPerSF,AskingRentGrowthYr,EffectiveRentPerUnit,EffectiveRentPerSF,EffectiveRentGrowthYr,VacancyUnits,...,RentType,AffordableType,ParkingSpacesUnit,ParkingSpaces,MarketSegment,latitude,longitude,Neighborhood,addr_clean,has heat pump
0,MBSEA272,2010 Q4,Existing,1245.6364,1.610721,0.042711,1242.4318,1.606577,0.042826,0.0000,...,Market,NaN,1.27,56.0,All,47.653136,-122.34682,FREMONT,3800 AURORA AVE N,No
1,MBSEA366,2000 Q1,Existing,NaN,NaN,NaN,NaN,NaN,NaN,1.1695,...,Market,NaN,NaN,0.0,All,47.558564,-122.38193,ALASKA JUNCTION,4821 FAUNTLEROY WAY SW,No
2,MBSEA536,2001 Q4,Existing,1324.5068,1.324507,0.054906,1311.2534,1.311253,0.054904,19.1916,...,Market,NaN,1.64,239.0,All,47.707318,-122.31922,NORTHGATE,10735 ROOSEVELT WAY NE,No
3,MBSEA353,2011 Q3,Existing,1527.6866,1.974225,0.068602,1516.6468,1.959958,0.067336,12.3010,...,Market,NaN,0.27,54.0,All,47.630066,-122.34263,QUEEN ANNE,12111215 DEXTER AVE N,No
4,MBSEA197,2023 Q4 QTD,Existing,NaN,NaN,NaN,NaN,NaN,NaN,2.6271,...,Market,NaN,NaN,NaN,All,47.658971,-122.34192,WALLINGFORD,1307 N 43RD ST,No


In [ ]:
data_trade = pd.read_csv('./01Data/Filtered Trade Data.csv')
###note that excel date starts from 1900-01-01, so need to convert it to normal date
data_trade['Applied Date'] = pd.to_datetime(data_trade['Applied Date'], origin='1899-12-30', unit='D')
data_trade['Expires Date'] = pd.to_datetime(data_trade['Expires Date'], origin='1899-12-30', unit='D')
data_trade['Completed Date'] = pd.to_datetime(data_trade['Completed Date'], origin='1899-12-30', unit='D')


## if Expires Date is NaN, then 'Issued'=='0' otherwise '1'
data_trade['Issued'] = 1- data_trade['Expires Date'].isna().astype(int)
data_trade['Issued'].value_counts()

## if Completed Date is NaN, then 'Completed'=='0' otherwise '1'
data_trade['Completed'] = 1- data_trade['Completed Date'].isna().astype(int)
data_trade['Completed'].value_counts() 



Issued
1    6326
0       8
Name: count, dtype: int64

Completed
1    5945
0     389
Name: count, dtype: int64

In [171]:

data_trade_all = pd.read_excel('./03ExportedData/all_trade_hp_addresses_20251003.xlsx')


In [ ]:
data_trade_all['Applied Date'] = pd.to_datetime(data_trade_all['AppliedDate'], origin='1899-12-30', unit='D')
data_trade_all['Expires Date'] = pd.to_datetime(data_trade_all['ExpiresDate'], origin='1899-12-30', unit='D')
data_trade_all['Completed Date'] = pd.to_datetime(data_trade_all['CompletedDate'], origin='1899-12-30', unit='D')  

### data_trade_all columns might look similar but not exactly the same as data_trade columns


In [196]:

data_trade_all.columns
data_trade.columns
## make a map of the columns from data_trade_all to data_trade
## apply new column names to data_trade_all

data_trade_all_renamed = data_trade_all.rename(columns={
    'PermitNum': 'Permit Number',
    'PermitClass': 'Permit Class',
    'PermitClassMapped': 'Permit Class Mapped',
    'PermitType': 'Perm',
    'EstProjectCost': 'Est Project Cost',
    'OriginalAddress1': 'Address ',
    'AppliedDate': 'Applied Date',
    'ExpiresDate': 'Expires Date',
    'CompletedDate': 'Completed Date',
    'TradeType': 'Trade Type',
    'Description': 'Description',
    'Link': 'Link',
    'OwnerName': 'Owner Name',
    'ContractorCompanyName': 'Contractor Name'
})

data_trade_all_renamed.head()

# data_trade_all_renamed.info()
data_trade_all.info()
## change the data_trade_all column names to data_trade column names


Index(['PermitNum', 'PermitClass', 'PermitClassMapped', 'PermitType',
       'PermitTypeMapped', 'Description', 'EstProjectCost', 'AppliedDate',
       'IssuedDate', 'ExpiresDate', 'CompletedDate', 'StatusCurrent',
       'OriginalAddress1', 'OriginalCity', 'OriginalState', 'OriginalZip',
       'ContractorCompanyName', 'Link', 'Latitude', 'Longitude', 'Location1',
       'addr_clean'],
      dtype='object')

Index(['Permit Number', 'Permit Class', 'Permit Class Mapped', 'Perm',
       'Permit Type Mapped', 'Description', 'Appliance Type',
       'Est Project Cost', 'Contractor Name', 'Applied Date', 'Issued Date',
       'Expires Date', 'Completed Date', 'Month Applied For',
       'Year Completed ', 'Current Status', 'PIN', 'Address ',
       'Property Name ', 'Detailed Existing Land Use ',
       'Planning Existing Land Use Code ', 'General Existing Land Use',
       'Ownership Type', 'Number of Buildings ', 'Number of Floors ', '20K ',
       'Existing Building Square Feet', 'Number of Units ',
       'Year Built Group ', 'Year Built ', 'Neighborhoods', 'Link', 'Issued',
       'Completed', 'addr_clean'],
      dtype='object')

,Permit Number,Permit Class,Permit Class Mapped,Perm,PermitTypeMapped,Description,Est Project Cost,Applied Date,IssuedDate,Expires Date,...,Address,OriginalCity,OriginalState,OriginalZip,Contractor Name,Link,Latitude,Longitude,Location1,addr_clean
0,6001084-RF,Single Family/Duplex,Residential,Refrigeration,Trade,INSTALL REFRIG TUBING (LINESET),NaN,7/16/2002,7/16/2002,1/16/2004,...,547 NE 130TH ST,SEATTLE,WA,98125.0,Universal Mechanical Services,https://services.seattle.gov/portal/customize/...,47.722897,-122.321340,"(47.7228967, -122.3213397)",547 ne 130th st
1,6001127-RF,Single Family/Duplex,Residential,Refrigeration,Trade,INSTALL A NEW A/C UNIT,NaN,7/16/2002,7/16/2002,1/16/2004,...,943 N 78TH ST,SEATTLE,WA,98103.0,ANCHOR HEATING,https://services.seattle.gov/portal/customize/...,47.685262,-122.345327,"(47.68526247, -122.34532741)",943 n 78th st
2,6001129-RF,Single Family/Duplex,Residential,Refrigeration,Trade,INSTALL NEW A/C UNIT,NaN,7/16/2002,7/16/2002,1/16/2004,...,4554 NE 89TH ST,SEATTLE,WA,98115.0,ANCHOR HEATING,https://services.seattle.gov/portal/customize/...,47.693325,-122.279285,"(47.69332452, -122.27928472)",4554 ne 89th st
3,6001157-RF,Commercial,Non-Residential,Refrigeration,Trade,INSTALL REFRIGERATION FOR WALK-IN COOLER,NaN,7/16/2002,7/16/2002,1/16/2004,...,6540 CALIFORNIA AVE SW,SEATTLE,WA,98136.0,MACDONALD-MILLER SERVICE INC,https://services.seattle.gov/portal/customize/...,47.543667,-122.386662,"(47.54366738, -122.38666162)",6540 california ave sw
4,6001287-RF,Single Family/Duplex,Residential,Refrigeration,Trade,INSTALL (2) A/C SYSTEMS,NaN,7/18/2002,7/18/2002,1/18/2004,...,836 36TH AVE E,SEATTLE,WA,98112.0,CARDINAL HEATING,https://services.seattle.gov/portal/customize/...,47.627135,-122.286045,"(47.62713468, -122.28604468)",836 36th ave e


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72062 entries, 0 to 72061
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PermitNum              72062 non-null  object 
 1   PermitClass            72036 non-null  object 
 2   PermitClassMapped      72036 non-null  object 
 3   PermitType             72062 non-null  object 
 4   PermitTypeMapped       72062 non-null  object 
 5   Description            72014 non-null  object 
 6   EstProjectCost         48905 non-null  float64
 7   AppliedDate            71551 non-null  object 
 8   IssuedDate             71566 non-null  object 
 9   ExpiresDate            71581 non-null  object 
 10  CompletedDate          56204 non-null  object 
 11  StatusCurrent          72062 non-null  object 
 12  OriginalAddress1       71991 non-null  object 
 13  OriginalCity           66305 non-null  object 
 14  OriginalState          66370 non-null  object 
 15  Or

In [200]:


## join the two dataframes on columns with same names, using outer join to see all differences

# data_trade_combined = pd.merge(data_trade, data_trade_all_renamed, on=['Permit Number','Permit Class','Permit Class Mapped','Perm'], how='outer', indicator=True)
# data_trade_combined.head()
# data_trade_combined['_merge'].value_counts()

## instead of merge, do append of the rows that are in data_trade_all_renamed but not in data_trade
data_trade_all_only = data_trade_all_renamed[~data_trade_all_renamed['Permit Number'].isin(data_trade['Permit Number'])]
data_trade_appended = pd.concat([data_trade, data_trade_all_only], ignore_index=True)
data_trade_appended.info()
data_trade_appended['Permit Number'].nunique()
data_trade_appended.to_csv('./03ExportedData/combined_trade_data_20251003.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72062 entries, 0 to 72061
Data columns (total 44 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Permit Number                     72062 non-null  object 
 1   Permit Class                      72036 non-null  object 
 2   Permit Class Mapped               72036 non-null  object 
 3   Perm                              72062 non-null  object 
 4   Permit Type Mapped                6334 non-null   object 
 5   Description                       72014 non-null  object 
 6   Appliance Type                    6334 non-null   object 
 7   Est Project Cost                  48905 non-null  float64
 8   Contractor Name                   67058 non-null  object 
 9   Applied Date                      71550 non-null  object 
 10  Issued Date                       6325 non-null   object 
 11  Expires Date                      71580 non-null  object 
 12  Comp

72062

In [ ]:

data_trade.head()
data_trade_all.head()

,Permit Number,Permit Class,Permit Class Mapped,Perm,Permit Type Mapped,Description,Appliance Type,Est Project Cost,Contractor Name,Applied Date,...,20K,Existing Building Square Feet,Number of Units,Year Built Group,Year Built,Neighborhoods,Link,Issued,Completed,addr_clean
0,6002307-RF,Multifamily,Residential,Refrigeration,Trade,"INSTALL 36 SPLIT SYSTEM ""MITSUBISHI"" (12) 1.0 ...",Heat Pump,NaN,PACIFIC AIR CONTROL INC,2002-08-12,...,Above 20k,52669.0,36.0,1990-2009,2002.0,Cascade,https://services.seattle.gov/portal/customize/...,1,1,3217 eastlake ave e
1,6008068-RF,Multifamily,Residential,Refrigeration,Trade,RUN REFRIGERANT LINES TO 11 HEAT PUMPS.,Heat Pump,NaN,C A C COMMERCIAL AIR CORP,2002-12-10,...,Above 20k,33754.0,35.0,1990-2009,2002.0,West Seattle,https://services.seattle.gov/portal/customize/...,1,1,4847 california ave sw
2,6008696-RF,Multifamily,Residential,Refrigeration,Trade,PIPING (1) 1 1-2 TON SPLIT SYSTEM,Heat Pump,NaN,PACIFIC AIR CONTROL INC,2002-12-23,...,Bellow 20k,15608.0,14.0,1948-1969,1968.0,Beacon Hill,https://services.seattle.gov/portal/customize/...,1,1,2356 15th ave s
3,6014900-RF,Multifamily,Residential,Refrigeration,Trade,3 REFRIGERATION LINESETS FOR COOLING ONLY SPLI...,Heat Pump,NaN,Universal Mechanical Services,2003-05-05,...,Above 20k,33523.0,19.0,1990-2009,2004.0,Queen Anne,https://services.seattle.gov/portal/customize/...,1,1,1414 1st ave w
4,6018775-RF,Multifamily,Residential,Refrigeration,Trade,FURNISH AND INSTALL DUCTLESS SPLIT A/C #4C,Heat Pump,NaN,Cardinal Heating & A/C,2003-07-18,...,Above 20k,32054.0,2.0,1900-1947,2023.0,Downtown,https://services.seattle.gov/portal/customize/...,1,1,210 3rd ave s


,PermitNum,PermitClass,PermitClassMapped,PermitType,PermitTypeMapped,Description,EstProjectCost,AppliedDate,IssuedDate,ExpiresDate,...,OriginalAddress1,OriginalCity,OriginalState,OriginalZip,ContractorCompanyName,Link,Latitude,Longitude,Location1,addr_clean
0,6001084-RF,Single Family/Duplex,Residential,Refrigeration,Trade,INSTALL REFRIG TUBING (LINESET),NaN,7/16/2002,7/16/2002,1/16/2004,...,547 NE 130TH ST,SEATTLE,WA,98125.0,Universal Mechanical Services,https://services.seattle.gov/portal/customize/...,47.722897,-122.321340,"(47.7228967, -122.3213397)",547 ne 130th st
1,6001127-RF,Single Family/Duplex,Residential,Refrigeration,Trade,INSTALL A NEW A/C UNIT,NaN,7/16/2002,7/16/2002,1/16/2004,...,943 N 78TH ST,SEATTLE,WA,98103.0,ANCHOR HEATING,https://services.seattle.gov/portal/customize/...,47.685262,-122.345327,"(47.68526247, -122.34532741)",943 n 78th st
2,6001129-RF,Single Family/Duplex,Residential,Refrigeration,Trade,INSTALL NEW A/C UNIT,NaN,7/16/2002,7/16/2002,1/16/2004,...,4554 NE 89TH ST,SEATTLE,WA,98115.0,ANCHOR HEATING,https://services.seattle.gov/portal/customize/...,47.693325,-122.279285,"(47.69332452, -122.27928472)",4554 ne 89th st
3,6001157-RF,Commercial,Non-Residential,Refrigeration,Trade,INSTALL REFRIGERATION FOR WALK-IN COOLER,NaN,7/16/2002,7/16/2002,1/16/2004,...,6540 CALIFORNIA AVE SW,SEATTLE,WA,98136.0,MACDONALD-MILLER SERVICE INC,https://services.seattle.gov/portal/customize/...,47.543667,-122.386662,"(47.54366738, -122.38666162)",6540 california ave sw
4,6001287-RF,Single Family/Duplex,Residential,Refrigeration,Trade,INSTALL (2) A/C SYSTEMS,NaN,7/18/2002,7/18/2002,1/18/2004,...,836 36TH AVE E,SEATTLE,WA,98112.0,CARDINAL HEATING,https://services.seattle.gov/portal/customize/...,47.627135,-122.286045,"(47.62713468, -122.28604468)",836 36th ave e


In [237]:
data_trade = pd.read_csv('./03ExportedData/combined_trade_data_20251003.csv')
### group by 'Property Name','Address' and see counts
data_trade.groupby(['Property Name ','Address ']).size().reset_index(name='counts').sort_values(by='counts', ascending=False)

###check if any 'Address ' has 'UNIT' or 'APT' in it
data_trade['Address '].str.contains('UNIT|UNIT |APT|APT ').value_counts()
data_trade.info()

C:\Users\nings\AppData\Local\Temp\ipykernel_6192\3893635280.py:1: DtypeWarning: Columns (4,6,10,13,15,18,19,20,21,22,25,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  data_trade = pd.read_csv('./03ExportedData/combined_trade_data_20251003.csv')


,Property Name,Address,counts
3720,JOYA TOWNHOMES CONDOMINIUM,2009 25TH AVE S,61
3665,HIGH POINT AT GRAHAM,3430 SW GRAHAM ST,51
3516,CLOVERDALE STREET,3801 S CLOVERDALE ST,31
3584,FAIRWAY ESTATES CONDOMINIUM,8003 SAND POINT WAY NE,30
4044,SEATTLE HOUSING AUTHORITY,7201 38TH AVE S,30
...,...,...,...
475,0,1522 C NW 61ST ST,1
476,0,1523 B NW 90TH ST,1
4360,triplex,8817 A MIDVALE AVE N,1
4361,triplex,9516 INTERLAKE AVE N,1


Address 
False    71991
Name: count, dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72062 entries, 0 to 72061
Data columns (total 44 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Permit Number                     72062 non-null  object 
 1   Permit Class                      72036 non-null  object 
 2   Permit Class Mapped               72036 non-null  object 
 3   Perm                              72062 non-null  object 
 4   Permit Type Mapped                6334 non-null   object 
 5   Description                       72014 non-null  object 
 6   Appliance Type                    6334 non-null   object 
 7   Est Project Cost                  48905 non-null  float64
 8   Contractor Name                   67058 non-null  object 
 9   Applied Date                      71550 non-null  object 
 10  Issued Date                       6325 non-null   object 
 11  Expires Date                      71580 non-null  object 
 12  Comp

In [202]:
# helper to clean address
def clean_addr(series):
    return series.astype(str).str.lower().str.strip().str.replace(r'[^0-9a-z\s]', '', regex=True).str.replace(r'\s+', ' ', regex=True)
    ### Remove all characters except lowercase letters, digits, and whitespace
data_apt_time['addr_clean'] = clean_addr(data_apt_time['Address'])
data_trade['addr_clean'] = clean_addr(data_trade['Address '])
print(data_trade['addr_clean'])

0           3217 eastlake ave e
1        4847 california ave sw
2               2356 15th ave s
3                1414 1st ave w
4                 210 3rd ave s
                  ...          
72057              219 15th ave
72058          1707 ne 117th st
72059          1709 ne 117th st
72060         8310 linden ave n
72061     1529 queen anne ave n
Name: addr_clean, Length: 72062, dtype: object


## join the apt data and the trade data

In [203]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Fuzzy-join data_trade -> data_apt_unique_address using addr_clean
# Uses fuzzywuzzy.process.extractOne with fuzz.token_sort_ratio (imports exist in later cells)

# Work on unique trade addresses to speed up matching
trade_addrs = data_trade['addr_clean'].astype(str).unique()
print(trade_addrs)
data_apt_unique_address = data_apt_time[['addr_clean', 'BuildingName', 'StarRating', 'Submarket', 'City', 'ZIPCode', 'Stories', 'YearBuilt', 'YearRenov', 'Class', 'TrueOwner', 'Units', 'latitude', 'longitude', 'Neighborhood', 'LEED', 'GreenRating', 'ParkingSpaces', 'Style']].drop_duplicates()
data_apt_unique_address.head()
data_apt_unique_address.to_excel('02ProcessedData/data_apt_unique_address.xlsx', index=False)

['3217 eastlake ave e' '4847 california ave sw' '2356 15th ave s' ...
 '1707 ne 117th st' '1709 ne 117th st' '8310 linden ave n']


,addr_clean,BuildingName,StarRating,Submarket,City,ZIPCode,Stories,YearBuilt,YearRenov,Class,TrueOwner,Units,latitude,longitude,Neighborhood,LEED,GreenRating,ParkingSpaces,Style
0,3800 aurora ave n,Northbridge Apartments,3,Fremont,Seattle,98103,4.0,1994.0,NaN,B,SSS Properties,44.0,47.653136,-122.34682,FREMONT,NaN,NaN,56.0,Mid-Rise
1,4821 fauntleroy way sw,Thunderbird Apartments,2,Fairmount Park,Seattle,98116,2.0,1957.0,NaN,B,1: Edwin N Spaunhurst | 2: John N Garner | 3: ...,28.0,47.558564,-122.38193,ALASKA JUNCTION,NaN,NaN,0.0,Low-Rise
2,10735 roosevelt way ne,Park at Northgate Apartment Homes,3,Maple Leaf,Seattle,98125,3.0,1967.0,1995.0,B,"Wallace Properties, Inc.",146.0,47.707318,-122.31922,NORTHGATE,NaN,NaN,239.0,Garden
3,12111215 dexter ave n,Dexter Lake Union,4,Westlake,Seattle,98109,11.0,2004.0,NaN,B,The Sobrato Organization,201.0,47.630066,-122.34263,QUEEN ANNE,NaN,NaN,54.0,Mid-Rise
4,1307 n 43rd st,Pickering Place Apartments,3,Wallingford,Seattle,98103,5.0,NaN,NaN,B,Burton Walls Investments,52.0,47.658971,-122.34192,WALLINGFORD,NaN,NaN,NaN,Low-Rise


In [204]:



matches = []
for a in trade_addrs:
    res = process.extractOne(a, data_apt_unique_address['addr_clean'], scorer=fuzz.token_sort_ratio)
    if res is None:
        matches.append((a, None, 0))
    else:
        # res can be (match, score) or (match, score, index) depending on input; take first two
        match_addr = res[0]
        match_score = res[1] if len(res) > 1 else 0
        matches.append((a, match_addr, match_score))

matches_df = pd.DataFrame(matches, columns=['addr_clean', 'match_addr', 'match_score'])

# Merge the match results back into data_trade (and keep original order/index)
data_trade_matched = data_trade.merge(matches_df, on='addr_clean', how='right')

# Optional: flag high-confidence matches (adjust threshold as needed)
threshold = 98
data_trade_matched['high_conf_match'] = data_trade_matched['match_score'] >= threshold

# Inspect results
data_trade_matched[['addr_clean', 'match_addr', 'match_score', 'high_conf_match','Permit Number']].head(20)

,addr_clean,match_addr,match_score,high_conf_match,Permit Number
0,3217 eastlake ave e,2851 eastlake ave e,89,False,6002307-RF
1,3217 eastlake ave e,2851 eastlake ave e,89,False,6862496-RF
2,3217 eastlake ave e,2851 eastlake ave e,89,False,6026618-RF
3,3217 eastlake ave e,2851 eastlake ave e,89,False,6041353-RF
4,3217 eastlake ave e,2851 eastlake ave e,89,False,6051565-RF
5,3217 eastlake ave e,2851 eastlake ave e,89,False,6080229-RF
6,3217 eastlake ave e,2851 eastlake ave e,89,False,6858576-RF
7,3217 eastlake ave e,2851 eastlake ave e,89,False,6858578-RF
8,3217 eastlake ave e,2851 eastlake ave e,89,False,6858580-RF
9,3217 eastlake ave e,2851 eastlake ave e,89,False,6858582-RF


In [238]:
# data_trade_matched.to_excel('02ProcessedData/data_trade_fuzzy_matched.xlsx', index=False)

data_trade_matched_filtered = data_trade_matched[data_trade_matched['high_conf_match']]
data_trade_matched_filtered.info() 
matches_df.head()


<class 'pandas.core.frame.DataFrame'>
Index: 982 entries, 110 to 71888
Data columns (total 47 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Permit Number                     982 non-null    object 
 1   Permit Class                      982 non-null    object 
 2   Permit Class Mapped               982 non-null    object 
 3   Perm                              982 non-null    object 
 4   Permit Type Mapped                163 non-null    object 
 5   Description                       982 non-null    object 
 6   Appliance Type                    163 non-null    object 
 7   Est Project Cost                  403 non-null    float64
 8   Contractor Name                   911 non-null    object 
 9   Applied Date                      974 non-null    object 
 10  Issued Date                       162 non-null    object 
 11  Expires Date                      974 non-null    object 
 12  Completed

,addr_clean,match_addr,match_score
0,3217 eastlake ave e,2851 eastlake ave e,89
1,4847 california ave sw,4730 california ave sw,91
2,2356 15th ave s,4351 15th ave s,87
3,1414 1st ave w,111 21st ave e,79
4,210 3rd ave s,140 23rd ave s,89


In [239]:


## merge the two datasets using data_apt_unique_address and _trade_matched_filtered on 'addr_clean' and 'match_addr'
data_merged = pd.merge(data_apt_unique_address, data_trade_matched_filtered, left_on='addr_clean', right_on='match_addr', how='inner')


In [240]:
data_merged.info()
data_merged['match_addr'].value_counts(dropna=False)
# data_merged.to_excel('02ProcessedData/data_merged_trade_aptmt_highconf.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 983 entries, 0 to 982
Data columns (total 66 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   addr_clean_x                      983 non-null    object 
 1   BuildingName                      972 non-null    object 
 2   StarRating                        983 non-null    int64  
 3   Submarket                         983 non-null    object 
 4   City                              983 non-null    object 
 5   ZIPCode                           983 non-null    int64  
 6   Stories                           982 non-null    float64
 7   YearBuilt                         980 non-null    float64
 8   YearRenov                         39 non-null     float64
 9   Class                             983 non-null    object 
 10  TrueOwner                         958 non-null    object 
 11  Units                             983 non-null    float64
 12  latitude

match_addr
900 university st         15
523 broadway e            12
4755 fauntleroy way sw    10
1700 e madison st          9
1620 broadway              9
                          ..
1700 17th ave              1
1008 n 109th st            1
2851 eastlake ave e        1
6105 roosevelt way ne      1
429 2nd ave w              1
Name: count, Length: 508, dtype: int64

In [241]:
### Check Alidu's code below
df = data_merged.copy()
## make Description column all upper case
df['Description'] = df['Description'].str.upper()
df.head()




,addr_clean_x,BuildingName,StarRating,Submarket,City,ZIPCode,Stories,YearBuilt,YearRenov,Class,...,StatusCurrent,OriginalCity,OriginalState,OriginalZip,Latitude,Longitude,Location1,match_addr,match_score,high_conf_match
0,2428 nw market st,AMLI Mark24,4,Ballard,Seattle,98107,6.0,2014.0,NaN,A,...,Completed,SEATTLE,WA,98107.0,47.668952,-122.388821,"(47.66895245, -122.38882126)",2428 nw market st,100,True
1,2428 nw market st,AMLI Mark24,4,Ballard,Seattle,98107,6.0,2014.0,NaN,A,...,Completed,SEATTLE,WA,98107.0,47.668952,-122.388821,"(47.66895245, -122.38882126)",2428 nw market st,100,True
2,2428 nw market st,AMLI Mark24,4,Ballard,Seattle,98107,6.0,2014.0,NaN,A,...,Completed,SEATTLE,WA,98107.0,47.668952,-122.388821,"(47.66895245, -122.38882126)",2428 nw market st,100,True
3,912 dexter ave n,Neptune,4,South Lake Union,Seattle,98109,6.0,2006.0,NaN,A,...,Completed,SEATTLE,WA,98109.0,47.627610,-122.342118,"(47.62761049, -122.34211852)",912 dexter ave n,100,True
4,912 dexter ave n,Neptune,4,South Lake Union,Seattle,98109,6.0,2006.0,NaN,A,...,Completed,SEATTLE,WA,98109.0,47.627610,-122.342118,"(47.62761049, -122.34211852)",912 dexter ave n,100,True


✅ 1. One-to-One Configuration
- In many residential ductless systems, each heat pump unit (especially mini-splits) has its own dedicated compressor.
- So yes, in these cases: 1 heat pump = 1 compressor.
❌ 2. Multi-Zone Systems
- Some systems use one outdoor unit with a single compressor to serve multiple indoor units.
- Example: A multi-zone ductless system might have 3 indoor heads but only 1 compressor.
🔁 3. Staged or Modular Systems
- In larger or commercial setups, a single heat pump system might include multiple compressors for staging, redundancy, or load balancing.
- So: 1 heat pump system ≠ 1 compressor—it could be 2 or more.

## Siman's code below


## criteria for heat pump and not common area
- if the same address appear for multiple times for heat pump --> and units are similar to either the buidling number (that need double check with the links...) or the units number --> considered not common area...
- if only a few records for one address, but there are multiple units inside the description --> extract the units and then 


In [209]:
##test
import re
tests = [
    "INSTALL DUCTLESS MINI SPLIT A/C",
    "INSTALL AC UNIT",
    "INSTALL A C UNIT",
    "INSTALL A-C UNIT",
    "INSTALL A/C UNIT",   # full-width slash
    "AC, COOLING ONLY",
    "UNSTALL NEW DUCTLESS A/C UNIT"

]

AC_TERMS = r"\b(ac|air\s*condition(?:er|ing)?|a\/c)\b"

for t in tests:
    print(bool(re.search(AC_TERMS, t, re.IGNORECASE | re.VERBOSE)))

True
True
False
False
True
True
True


In [210]:
tests = [
    "INSTALL HP 2 ZONE WITH 2 INDOOR UNIT",
    "INSTALL 12K DHP"

]

HP_TERMS = r"""
\b(
    heat\s*pumps?               |   # heat pump / heat pumps
    (d)?h[\/\.]?p[\/\.]?        |   # h/p, h.p., hp
    mini[-\s]*splits?           |   # mini split / mini-splits
    split\s*systems?            |   # split system(s)
    ductless(?:\s*mini)?        |   # ductless / ductless mini
    mitsubishi
)\b
"""
for t in tests:
    print(bool(re.search(HP_TERMS, t, re.IGNORECASE | re.VERBOSE)))

True
True


In [249]:
### make a new column that to decide whether the permit is related to heat pump or not
# pattern = r'(?i)(?:(?!AC|air condition|a/c|piping|pipe).)*(heat pump(s)?|h/p|h\.p\.|hp|heatpump|mini split|mini-split|split system(s)|ductless min?)'
# pattern = r'(?i)^(?!.*(AC|air condition|a/c)).*(heat pump|h/p|H\/P|h\.p\.|hp|heatpump|minisplit|mini split|mini-split|split system|ductless min|MITSUBISHI|ductless head(s))' ## add MITSUBISHI and remove piping ... 
## though ductless head can be both AC and HP, 
pattern = r'(?!.*(AC|air condition|a/c(?!.*(HVAC|MACHINE|REPLACING|REPLACE|EACH)))).*(heat pump|h/p|H\/P|h\.p\.|hp|heatpump|minisplit|mini split|mini-split|split system|ductless min|MITSUBISHI|ductless head(s)?)' ## add MITSUBISHI and remove piping ... 
###another version

# 1) Define reusable regexes
HP_TERMS = r"""
\b(
    heat\s*pumps?               |   # heat pump / heat pumps
    (d)?h[\/\.]?p[\/\.]?        |   # h/p, h.p., hp
    mini[-\s]*splits?           |   # mini split / mini-splits
    split\s*systems?            |   # split system(s)
    ductless(?:\s*mini)?        |   # ductless / ductless mini
     mitsubishi
)\b
"""

LINE_TERMS = r"""
\b(
    refrig(?:erant)?\s*line(?:s|sets)? |   # refrigerant line(s)/linesets
    refer\s*lines?                      |   # 'refer lines' (common shorthand)
    piping
)\b
"""

AC_TERMS = r"\b((ductless*)?ac|air\s*condition(?:er|ing)?|a\/c)\b"

# 2) Compute helper booleans
has_hp_terms   = df["Description"].str.contains(HP_TERMS,   case=False, na=False, regex=True, flags=re.VERBOSE)
has_line_terms = df["Description"].str.contains(LINE_TERMS, case=False, na=False, regex=True, flags=re.VERBOSE)
has_ac_terms   = df["Description"].str.contains(AC_TERMS,   case=False, na=False, regex=True)

# 3) AC-only veto: it's AC-only if it mentions AC but NONE of the HP terms
ac_only = has_ac_terms & ~has_hp_terms


# 4) Final label:
#    - match if it has HP terms (covers "…HEAT PUMPS", "…SPLIT SYSTEMS", "MITSUBISHI", etc.)
#    - OR if it mentions line/piping terms together with HP terms somewhere in the same line
df["HP_related"] = ((has_hp_terms | (ac_only & has_hp_terms)) & ~ac_only).astype(int)

# df["HP_related"] = ( ~ac_only).astype(int)
sum(has_hp_terms)
sum((has_line_terms & has_hp_terms))
sum(ac_only)


#    r'heat pump in all units', r'hp in all units', r'hp for all units', r'in all units', r'in each unit',
    # r'each unit', r'every unit', r'in\-unit', r'apartment units', r'apt units', r'multi[- ]?zone', r'split heat pump',
    # r'mini split', r'multiple split systems', r'split systems', r'split system'

# df['HP_related'] = df['Description'].str.contains(pattern, case=False, na=False).astype(int)
df['HP_related'].value_counts()

## check out the head of those HP_related  == 0
df[df['HP_related']==0].head(5)
df[df['HP_related']==0]['addr_clean_x'].value_counts()

##write the df to excel
df.to_excel('02ProcessedData/merged_apt_trade.xlsx', index=False)

C:\Users\nings\AppData\Local\Temp\ipykernel_6192\3422441806.py:31: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  has_hp_terms   = df["Description"].str.contains(HP_TERMS,   case=False, na=False, regex=True, flags=re.VERBOSE)
C:\Users\nings\AppData\Local\Temp\ipykernel_6192\3422441806.py:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  has_line_terms = df["Description"].str.contains(LINE_TERMS, case=False, na=False, regex=True, flags=re.VERBOSE)
C:\Users\nings\AppData\Local\Temp\ipykernel_6192\3422441806.py:33: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  has_ac_terms   = df["Description"].str.contains(AC_TERMS,   case=False, na=False, regex=True)


594

239

32

HP_related
1    594
0    389
Name: count, dtype: int64

,addr_clean_x,BuildingName,StarRating,Submarket,City,ZIPCode,Stories,YearBuilt,YearRenov,Class,...,OriginalZip,Latitude,Longitude,Location1,match_addr,match_score,high_conf_match,Number,FiveWords,HP_related
2,2428 nw market st,AMLI Mark24,4,Ballard,Seattle,98107,6.0,2014.0,NaN,A,...,98107.0,47.668952,-122.388821,"(47.66895245, -122.38882126)",2428 nw market st,100,True,NaN,NaN,0
3,912 dexter ave n,Neptune,4,South Lake Union,Seattle,98109,6.0,2006.0,NaN,A,...,98109.0,47.627610,-122.342118,"(47.62761049, -122.34211852)",912 dexter ave n,100,True,19,SPLIT SYS REF. PIPING,0
4,912 dexter ave n,Neptune,4,South Lake Union,Seattle,98109,6.0,2006.0,NaN,A,...,98109.0,47.627610,-122.342118,"(47.62761049, -122.34211852)",912 dexter ave n,100,True,NaN,NaN,0
5,912 dexter ave n,Neptune,4,South Lake Union,Seattle,98109,6.0,2006.0,NaN,A,...,98109.0,47.627610,-122.342118,"(47.62761049, -122.34211852)",912 dexter ave n,100,True,NaN,NaN,0
6,651 s main st,Nihonmachi Terrace,4,Chinatown,Seattle,98104,6.0,2006.0,NaN,B,...,98104.0,47.599873,-122.324690,"(47.59987304, -122.32469042)",651 s main st,100,True,NaN,NaN,0


addr_clean_x
900 university st     11
1700 e madison st      7
1630 43rd ave e        6
523 broadway e         6
1900 1st ave           6
                      ..
7357 43rd ave s        1
800 columbia st        1
110 10th ave s         1
1170 republican st     1
1529 4th ave w         1
Name: count, Length: 247, dtype: int64

## Sensitivity Test
- if only do heat pump -- 3626
- if heat pump + mini split +  mitsubishi... 6295
- if heat pump +  mitsubishi ....3883

In [250]:
### check how many unique addresses in the df[df['HP_related']==1] 

# and how many unique addresses are in the df[df['HP_related']==0] 

df[df['HP_related']==1]['addr_clean_x'].nunique()
df[df['HP_related']==0]['addr_clean_x'].nunique()
df['addr_clean_x'].nunique()
df.describe(include='object').T

388

247

508

,count,unique,top,freq
addr_clean_x,983,508,900 university st,15
BuildingName,972,501,Horizon House,15
Submarket,983,62,Capitol Hill,128
City,983,1,Seattle,983
Class,983,2,A,596
TrueOwner,958,324,Equity Residential,28
Neighborhood,983,46,CAPITOL HILL,130
GreenRating,150,8,Silver,55
Style,981,4,Mid-Rise,821
Permit Number,983,982,6775772-RF,2


In [279]:
## extract any numbers and at least 5 words after from the 'Description' column
# df['Description_Number'] = df['Description'].str.extract(r'(\d+).*?(\b\w+\b.*?){5}', expand=True)

# pattern = r'\(?(\d+)\)?\D+((?:\b\w+\b\W+){2,})'
# pattern = r'\(?(\d+)\)?.*?\b(\w+\b(?:\W+\w+\b){2,})'
# pattern = r'\(?(\d+)\)?\D+((?:\b\w+\b\W+){2,})'
# pattern = r'\(?(\d+)\)?\s+(.*?\b\w+\b(?:\W+\b\w+\b){1,})'  ##but not before 'FAN COIL UNITS'
pattern = r'\(?(\d+)\)?\s+(?!FAN COIL UNITS)(.*?\b\w+\b(?:\W+\b\w+\b){1,}|\[\d\])'

# pattern = r'(?i)(?!.*\b(?:AC|air condition|a/c|piping|pipe)\b).*?\(?(\d+)\)?\s*(?:\w+\s*){0,3}?\b(heat pump|h/p|h\.p\.|hp|heatpump|mini split|mini-split|split system|ductless min)\b'


df[['Number', 'FiveWords']] = df['Description'].str.extract(
    pattern,
    expand=True)

df[['Description', 'Number', 'FiveWords']].head(20)
df.info()
df.describe(include='object').T

,Description,Number,FiveWords
0,REFRIGERATION PIPING FOR 8 SPLIT SYSTEM HEAT P...,8,SPLIT SYSTEM HEAT PUMPS
1,INSTALLATION OF ONE SPLIT SYSTEM HEAT PUMP.,NaN,NaN
2,REFRIGERATION PIPING FOR TWO NEW AIR CONDITION...,NaN,NaN
3,INSTALL 19 SPLIT SYS REF. PIPING,19,SPLIT SYS REF. PIPING
4,PIPING FOR WALK IN COOLER,NaN,NaN
5,INSTALLING REFRIGERATION LINES,NaN,NaN
6,INSTALL REFER LINE TO A/C UNIT,NaN,NaN
7,INSTALLING REFER LINE TO UNIT,NaN,NaN
8,INSTALL [9] SPLIT SYSTEMS. 3/7/16 JCB; ADDRESS...,16,JCB; ADDRESS CHANGED FROM 3400 WALLINGFORD AVE...
9,REPLACE A/C IN ELEVATOR MACHINE ROOM,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 983 entries, 0 to 982
Data columns (total 69 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   addr_clean_x                     983 non-null    object 
 1   BuildingName                     972 non-null    object 
 2   StarRating                       983 non-null    int64  
 3   Submarket                        983 non-null    object 
 4   City                             983 non-null    object 
 5   ZIPCode                          983 non-null    int64  
 6   Stories                          982 non-null    float64
 7   YearBuilt                        980 non-null    float64
 8   YearRenov                        39 non-null     float64
 9   Class                            983 non-null    object 
 10  TrueOwner                        958 non-null    object 
 11  Units                            983 non-null    float64
 12  latitude              

,count,unique,top,freq
addr_clean_x,983,508,900 university st,15
BuildingName,972,501,Horizon House,15
Submarket,983,62,Capitol Hill,128
City,983,1,Seattle,983
Class,983,2,A,596
TrueOwner,958,324,Equity Residential,28
Neighborhood,983,46,CAPITOL HILL,130
GreenRating,150,8,Silver,55
Style,981,4,Mid-Rise,821
Permit Number,983,982,6775772-RF,2


In [277]:
matches = df['Description'].str.extractall(
    r'\((\d+)\)\s*((?:[\d\-\.]+\s*){1,2}(?:\w+\s*){1,5})'
)

matches = matches.reset_index()
numbers = matches.pivot(index='level_0', columns='match', values=0)
texts = matches.pivot(index='level_0', columns='match', values=1)

numbers.columns = [f'TNumber{i+1}' for i in range(len(numbers.columns))]
texts.columns = [f'Text{i+1}' for i in range(len(texts.columns))]

df_combined = df.join(numbers).join(texts)
## sum up all the Number columns to see if there is any record with multiple units (if number column is NAN then it is 0)


number_cols = [col for col in df_combined.columns if col.startswith('TNumber')]

df_combined[number_cols] = df_combined[number_cols].apply(
    pd.to_numeric, errors='coerce'
)

df_combined['Number_sum'] = df_combined[number_cols].fillna(0).sum(axis=1)
df_combined['Number_final'] = np.where(df_combined['Number_sum']>0, df_combined['Number_sum'], df_combined['Number'])

df_combined.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 983 entries, 0 to 982
Data columns (total 77 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   addr_clean_x                     983 non-null    object 
 1   BuildingName                     972 non-null    object 
 2   StarRating                       983 non-null    int64  
 3   Submarket                        983 non-null    object 
 4   City                             983 non-null    object 
 5   ZIPCode                          983 non-null    int64  
 6   Stories                          982 non-null    float64
 7   YearBuilt                        980 non-null    float64
 8   YearRenov                        39 non-null     float64
 9   Class                            983 non-null    object 
 10  TrueOwner                        958 non-null    object 
 11  Units                            983 non-null    float64
 12  latitude              

In [283]:
matches = df['Description'].str.extractall(
    r'\((\d+)\)\s*((?:[\d\-\.]+\s*){1,2}(?:\w+\s*){1,5})'
)

matches = matches.reset_index()
numbers = matches.pivot(index='level_0', columns='match', values=0)
texts = matches.pivot(index='level_0', columns='match', values=1)

numbers.columns = [f'TNumber{i+1}' for i in range(len(numbers.columns))]
texts.columns = [f'Text{i+1}' for i in range(len(texts.columns))]

df_combined = df.join(numbers).join(texts)
## sum up all the Number columns to see if there is any record with multiple units (if number column is NAN then it is 0)


number_cols = [col for col in df_combined.columns if col.startswith('TNumber')]

df_combined[number_cols] = df_combined[number_cols].apply(
    pd.to_numeric, errors='coerce'
)

df_combined['Number_sum'] = df_combined[number_cols].fillna(0).sum(axis=1)
df_combined['Number_final'] = np.where(df_combined['Number_sum']>0, df_combined['Number_sum'], df_combined['Number'])

df_combined.head(20)
df_combined.info()
df_combined.describe(include='object').T

,addr_clean_x,BuildingName,StarRating,Submarket,City,ZIPCode,Stories,YearBuilt,YearRenov,Class,TrueOwner,Units,latitude,longitude,Neighborhood,LEED,GreenRating,ParkingSpaces,Style,Permit Number,Permit Class,Permit Class Mapped,Perm,Permit Type Mapped,Description,Appliance Type,Est Project Cost,Contractor Name,Applied Date,Issued Date,Expires Date,Completed Date,Month Applied For,Year Completed,Current Status,PIN,Address,Property Name,Detailed Existing Land Use,Planning Existing Land Use Code,General Existing Land Use,Ownership Type,Number of Buildings,Number of Floors,20K,Existing Building Square Feet,Number of Units,Year Built Group,Year Built,Neighborhoods,Link,Issued,Completed,addr_clean_y,PermitTypeMapped,IssuedDate,StatusCurrent,OriginalCity,OriginalState,OriginalZip,Latitude,Longitude,Location1,match_addr,match_score,high_conf_match,Number,FiveWords,HP_related,TNumber1,TNumber2,TNumber3,Text1,Text2,Text3,Number_sum,Number_final
0,2428 nw market st,AMLI Mark24,4,Ballard,Seattle,98107,6.0,2014.0,NaN,A,AMLI Management Company,304.0,47.668879,-122.38886,BALLARD SOUTH,1.0,platinum,120.0,Mid-Rise,6360063-RF,Commercial,Non-Residential,Refrigeration,NaN,REFRIGERATION PIPING FOR 8 SPLIT SYSTEM HEAT P...,NaN,NaN,MERIT MECHANICAL INC,4/5/2013,NaN,10/5/2014,8/13/2014,NaN,NaN,NaN,NaN,2428 NW MARKET ST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://services.seattle.gov/portal/customize/...,NaN,NaN,2428 nw market st,Trade,4/5/2013,Completed,SEATTLE,WA,98107.0,47.668952,-122.388821,"(47.66895245, -122.38882126)",2428 nw market st,100,True,8,SPLIT SYSTEM HEAT PUMPS,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,8
1,2428 nw market st,AMLI Mark24,4,Ballard,Seattle,98107,6.0,2014.0,NaN,A,AMLI Management Company,304.0,47.668879,-122.38886,BALLARD SOUTH,1.0,platinum,120.0,Mid-Rise,6448842-RF,Single Family/Duplex,Residential,Refrigeration,NaN,INSTALLATION OF ONE SPLIT SYSTEM HEAT PUMP.,NaN,NaN,EVERGREEN REFRIGERATION LLC,12/4/2014,NaN,6/4/2016,1/30/2015,NaN,NaN,NaN,NaN,2428 NW MARKET ST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://services.seattle.gov/portal/customize/...,NaN,NaN,2428 nw market st,Trade,12/4/2014,Completed,SEATTLE,WA,98107.0,47.668952,-122.388821,"(47.66895245, -122.38882126)",2428 nw market st,100,True,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,2428 nw market st,AMLI Mark24,4,Ballard,Seattle,98107,6.0,2014.0,NaN,A,AMLI Management Company,304.0,47.668879,-122.38886,BALLARD SOUTH,1.0,platinum,120.0,Mid-Rise,6671391-RF,Commercial,Non-Residential,Refrigeration,NaN,REFRIGERATION PIPING FOR TWO NEW AIR CONDITION...,NaN,6500.0,UNITED SYSTEMS MECHANICAL LLC,6/5/2018,NaN,12/5/2019,6/6/2018,NaN,NaN,NaN,NaN,2428 NW MARKET ST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://services.seattle.gov/portal/customize/...,NaN,NaN,2428 nw market st,Trade,6/5/2018,Completed,SEATTLE,WA,98107.0,47.668952,-122.388821,"(47.66895245, -122.38882126)",2428 nw market st,100,True,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,912 dexter ave n,Neptune,4,South Lake Union,Seattle,98109,6.0,2006.0,NaN,A,1: Domain Capital Group | 2: PFA Ejendomme A/S...,234.0,47.627541,-122.34208,QUEEN ANNE,NaN,NaN,714.0,Mid-Rise,6091968-RF,Commercial,Non-Residential,Refrigeration,NaN,INSTALL 19 SPLIT SYS REF. PIPING,NaN,NaN,PRO TECH REFRIGERATION LLC,3/20/2006,NaN,9/20/2007,8/2/2006,NaN,NaN,NaN,NaN,912 DEXTER AVE N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://services.seattle.gov/portal/customize/...,NaN,NaN,912 dexter ave n,Trade,3/20/2006,Completed,SEATTLE,WA,98109.0,47.627610,-122.342118,"(47.62761049, -122.34211852)",912 dexter ave n,100,True,19,SPLIT SYS REF. PIPING,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,19
4,912 dexter ave n,Neptune,4,South Lake Union,Seattle,98109,6.0,2006.0,NaN,A,1: Domain Capital Group | 2: PFA Ejendomme A/S...,234.0,47.627541,-122.34208,QUEEN ANNE,NaN,NaN,714.0,Mid-Rise,6175558-RF,Commercial,Non-Residential,Refrigeration,NaN,PIPING FOR WALK IN COOLER,NaN,NaN,NaN,3/28/2008,NaN,9/28/2009,7/7/2008,NaN,NaN,NaN,NaN,912 DEXTER AV

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 983 entries, 0 to 982
Data columns (total 77 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   addr_clean_x                     983 non-null    object 
 1   BuildingName                     972 non-null    object 
 2   StarRating                       983 non-null    int64  
 3   Submarket                        983 non-null    object 
 4   City                             983 non-null    object 
 5   ZIPCode                          983 non-null    int64  
 6   Stories                          982 non-null    float64
 7   YearBuilt                        980 non-null    float64
 8   YearRenov                        39 non-null     float64
 9   Class                            983 non-null    object 
 10  TrueOwner                        958 non-null    object 
 11  Units                            983 non-null    float64
 12  latitude              

,count,unique,top,freq
addr_clean_x,983,508,900 university st,15
BuildingName,972,501,Horizon House,15
Submarket,983,62,Capitol Hill,128
City,983,1,Seattle,983
Class,983,2,A,596
TrueOwner,958,324,Equity Residential,28
Neighborhood,983,46,CAPITOL HILL,130
GreenRating,150,8,Silver,55
Style,981,4,Mid-Rise,821
Permit Number,983,982,6775772-RF,2


In [216]:

# ### now only extract numbers that are within ()
# # 
# # pattern = r'\((\d+(?:\.\d+)?)\)\s*([^\d()]+)'
# pattern = r'\((\d+)\)\s*([\d\.]+\s*\w+(?:\s*\w+)*)'
# pattern = r'\((\d+)\)\s*((?:[\d\-\.]+\s*){1,2}(?:\w+\s*){1,3})'
# matches = df['Description'].str.extractall(pattern)

# matches = matches.reset_index()
# pivoted = matches.pivot(index='level_0', columns='match', values=0)
# pivoted.columns = ['Number1', 'Number2']  # adjust as needed  ##, 'Number3', 'Number4', 'Number5'

# texts = matches.pivot(index='level_0', columns='match', values=1)
# texts.columns = ['Text1', 'Text2'] ##, 'Text3', 'Text4', 'Text5'

# df_combined = df.join(pivoted).join(texts)

# df_combined[df_combined['Number2'].notna()][['Description', 'Number1', 'Text1', 'Number2', 'Text2']].head(20) ##, 'Number3', 'Text3', 'Number4', 'Text4', 'Number5', 'Text5'

In [ ]:
##filter by HP_related == 1 and then group by 'Detailed Existing Land Use','Year Completed', 'Address','Number of Units','Number of Floors', 'Number of Buildings'
# df['Number of Floors'] = df['Number of Floors'].fillna(-1)
# df['Number of Buildings'] = df['Number of Buildings'].fillna(-1)
# df['Number of Units'] = df['Number of Units'].fillna(-1)  

df.columns = df.columns.str.strip()  ## remove any leading or trailing spaces in column names

## get the year of completion by extract year from 'Completed Date' (if not available, then from 'Expires Date') using ifelse
df_combined['Year Completed'] =  df_combined['Completed Date'].astype(str).str.extract(r'(\d{4})').astype(float)
## get count of records by 'Year Completed', 'addr_clean_x', 'Permit Class', 'Number of Buildings','Units', 'Stories'
# t = df_combined.groupby(['Year Completed', 'addr_clean_x', 'Permit Class']).agg({'Number of Buildings':'first', 'Units':'first', 'Stories':'first'}).reset_index()  
t.head()
t.info()
df_sum = df_combined.groupby(['Year Completed', 'addr_clean_x', 'Permit Class',
                                            'Units', 'Stories']).size().reset_index(name='n').sort_values(by='n', ascending=False) 
##,'Number of Units','Number of Floors'
df_sum['n_unit_ratio'] = abs(df_sum['n']/df_sum['Units'])
df_sum['n_story_ratio'] = abs(df_sum['n']/df_sum['Stories'])

# df_sum['n_building_ratio'] = abs(df_sum['n']/df_sum['Number of Buildings'])


df_sum = df_sum.sort_values(by=['n_unit_ratio','n_story_ratio'], ascending= [False, False]) #, 'n_building_ratio'
df_sum.head(20)

df_sum.describe().T
df_sum.info()


,Year Completed,addr_clean_x,Permit Class,Number of Buildings,Units,Stories
0,2014.0,100 denny way,Commercial,NaN,164.0,7.0
1,2014.0,100 denny way,Multifamily,1.0,164.0,7.0
2,2014.0,100 taylor ave n,Commercial,NaN,197.0,6.0
3,2014.0,1000 8th ave,Commercial,NaN,358.0,14.0
4,2014.0,1000 8th ave,Multifamily,3.0,358.0,14.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year Completed       598 non-null    float64
 1   addr_clean_x         598 non-null    object 
 2   Permit Class         598 non-null    object 
 3   Number of Buildings  143 non-null    float64
 4   Units                598 non-null    float64
 5   Stories              597 non-null    float64
dtypes: float64(4), object(2)
memory usage: 28.2+ KB


,Year Completed,addr_clean_x,Permit Class,Units,Stories,n,n_unit_ratio,n_story_ratio
313,2016.0,5000 30th ave ne,Multifamily,16.0,3.0,3,0.187500,1.000000
337,2017.0,1240 n midvale pl,Commercial,30.0,4.0,4,0.133333,1.000000
685,2023.0,7009 greenwood ave n,Commercial,36.0,5.0,4,0.111111,0.800000
686,2023.0,7009 greenwood ave n,Multifamily,36.0,5.0,3,0.083333,0.600000
647,2022.0,7006 greenwood ave n,Multifamily,25.0,2.0,2,0.080000,1.000000
8,2004.0,1701 n 45th st,Commercial,25.0,3.0,2,0.080000,0.666667
465,2019.0,1307 harbor ave sw,Commercial,15.0,7.0,1,0.066667,0.142857
281,2016.0,1900 1st ave,Commercial,93.0,11.0,6,0.064516,0.545455
267,2016.0,1315 e jefferson st,Commercial,32.0,4.0,2,0.062500,0.500000
411,2018.0,1315 e jefferson st,Commercial,32.0,4.0,2,0.062500,0.500000


,count,mean,std,min,25%,50%,75%,max
Year Completed,723.0,2016.319502,4.988796,2003.000000,2014.000000,2017.000000,2020.000000,2025.0000
Units,723.0,143.619640,111.213580,15.000000,59.000000,106.000000,208.000000,761.0000
Stories,723.0,7.448133,6.279590,1.000000,4.000000,6.000000,7.000000,44.0000
n,723.0,1.179806,0.534962,1.000000,1.000000,1.000000,1.000000,6.0000
n_unit_ratio,723.0,0.014483,0.014606,0.001314,0.005602,0.010417,0.018692,0.1875
n_story_ratio,723.0,0.212933,0.136539,0.022727,0.142857,0.166667,0.250000,1.0000


<class 'pandas.core.frame.DataFrame'>
Index: 723 entries, 313 to 463
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Year Completed  723 non-null    float64
 1   addr_clean_x    723 non-null    object 
 2   Permit Class    723 non-null    object 
 3   Units           723 non-null    float64
 4   Stories         723 non-null    float64
 5   n               723 non-null    int64  
 6   n_unit_ratio    723 non-null    float64
 7   n_story_ratio   723 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 50.8+ KB


In [357]:
### join df_combined with df_sum to get the final dataframe

df_combined.columns = df_combined.columns.str.strip()  ## remove any leading or trailing spaces in column names
df_final = df_combined.merge(df_sum, on=[ 'Year Completed','addr_clean_x'], how='left', suffixes=('', '_y'))
df_final.head(5)
df_final.describe()
df_final.info()


,addr_clean_x,BuildingName,StarRating,Submarket,City,ZIPCode,Stories,YearBuilt,YearRenov,Class,TrueOwner,Units,latitude,longitude,Neighborhood,LEED,GreenRating,ParkingSpaces,Style,Permit Number,Permit Class,Permit Class Mapped,Perm,Permit Type Mapped,Description,Appliance Type,Est Project Cost,Contractor Name,Applied Date,Issued Date,Expires Date,Completed Date,Month Applied For,Year Completed,Current Status,PIN,Address,Property Name,Detailed Existing Land Use,Planning Existing Land Use Code,General Existing Land Use,Ownership Type,Number of Buildings,Number of Floors,20K,Existing Building Square Feet,Number of Units,Year Built Group,Year Built,Neighborhoods,Link,Issued,Completed,addr_clean_y,PermitTypeMapped,IssuedDate,StatusCurrent,OriginalCity,OriginalState,OriginalZip,Latitude,Longitude,Location1,match_addr,match_score,high_conf_match,Number,FiveWords,HP_related,TNumber1,TNumber2,TNumber3,Text1,Text2,Text3,Number_sum,Number_final,Permit Class_y,Units_y,Stories_y,n,n_unit_ratio,n_story_ratio
0,2428 nw market st,AMLI Mark24,4,Ballard,Seattle,98107,6.0,2014.0,NaN,A,AMLI Management Company,304.0,47.668879,-122.38886,BALLARD SOUTH,1.0,platinum,120.0,Mid-Rise,6360063-RF,Commercial,Non-Residential,Refrigeration,NaN,REFRIGERATION PIPING FOR 8 SPLIT SYSTEM HEAT P...,NaN,NaN,MERIT MECHANICAL INC,4/5/2013,NaN,10/5/2014,8/13/2014,NaN,2014.0,NaN,NaN,2428 NW MARKET ST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://services.seattle.gov/portal/customize/...,NaN,NaN,2428 nw market st,Trade,4/5/2013,Completed,SEATTLE,WA,98107.0,47.668952,-122.388821,"(47.66895245, -122.38882126)",2428 nw market st,100,True,8,SPLIT SYSTEM HEAT PUMPS,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,8,Commercial,304.0,6.0,1.0,0.003289,0.166667
1,2428 nw market st,AMLI Mark24,4,Ballard,Seattle,98107,6.0,2014.0,NaN,A,AMLI Management Company,304.0,47.668879,-122.38886,BALLARD SOUTH,1.0,platinum,120.0,Mid-Rise,6448842-RF,Single Family/Duplex,Residential,Refrigeration,NaN,INSTALLATION OF ONE SPLIT SYSTEM HEAT PUMP.,NaN,NaN,EVERGREEN REFRIGERATION LLC,12/4/2014,NaN,6/4/2016,1/30/2015,NaN,2015.0,NaN,NaN,2428 NW MARKET ST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://services.seattle.gov/portal/customize/...,NaN,NaN,2428 nw market st,Trade,12/4/2014,Completed,SEATTLE,WA,98107.0,47.668952,-122.388821,"(47.66895245, -122.38882126)",2428 nw market st,100,True,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Single Family/Duplex,304.0,6.0,1.0,0.003289,0.166667
2,2428 nw market st,AMLI Mark24,4,Ballard,Seattle,98107,6.0,2014.0,NaN,A,AMLI Management Company,304.0,47.668879,-122.38886,BALLARD SOUTH,1.0,platinum,120.0,Mid-Rise,6671391-RF,Commercial,Non-Residential,Refrigeration,NaN,REFRIGERATION PIPING FOR TWO NEW AIR CONDITION...,NaN,6500.0,UNITED SYSTEMS MECHANICAL LLC,6/5/2018,NaN,12/5/2019,6/6/2018,NaN,2018.0,NaN,NaN,2428 NW MARKET ST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://services.seattle.gov/portal/customize/...,NaN,NaN,2428 nw market st,Trade,6/5/2018,Completed,SEATTLE,WA,98107.0,47.668952,-122.388821,"(47.66895245, -122.38882126)",2428 nw market st,100,True,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Commercial,304.0,6.0,1.0,0.003289,0.166667
3,912 dexter ave n,Neptune,4,South Lake Union,Seattle,98109,6.0,2006.0,NaN,A,1: Domain Capital Group | 2: PFA Ejendomme A/S...,234.0,47.627541,-122.34208,QUEEN ANNE,NaN,NaN,714.0,Mid-Rise,6091968-RF,Commercial,Non-Residential,Refrigeration,NaN,INSTALL 19 SPLIT SYS REF. PIPING,NaN,NaN,PRO TECH REFRIGERATION LLC,3/20/2006,NaN,9/20/2007,8/2/2006,NaN,2006.0,NaN,NaN,912 DEXTER AVE N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://services.seattle.gov/portal/customize/...,NaN,NaN,912 dexter ave n,Trade,3/20/2006,Completed,SEATTLE,WA,98109.0,47.627610,-122.342118,"(47.62761049, -122.34211852)",912 dexter ave n,100,True,19,SPLIT SYS REF. PIPING,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,19,Commercial,234.0,6.0,1.0,0.004274,0.166667
4,912 dexter ave n,Neptune,4,South Lake Union,Seattle,98109,6.0,2006.0,NaN,A,1: Domain C

,StarRating,ZIPCode,Stories,YearBuilt,YearRenov,Units,latitude,longitude,LEED,ParkingSpaces,Est Project Cost,Year Completed,PIN,Number of Buildings,Number of Floors,Existing Building Square Feet,Number of Units,Year Built,Issued,Completed,OriginalZip,Latitude,Longitude,match_score,HP_related,TNumber1,TNumber2,TNumber3,Number_sum,Units_y,Stories_y,n,n_unit_ratio,n_story_ratio
count,1042.000000,1042.000000,1041.000000,1039.000000,43.000000,1042.000000,1042.000000,1042.000000,157.0,546.000000,4.260000e+02,913.000000,1.760000e+02,172.000000,172.000000,172.000000,172.000000,172.000000,176.000000,176.000000,834.000000,866.000000,866.000000,1042.000000,1042.000000,4.400000e+01,15.000000,2.0,1.042000e+03,912.000000,912.000000,912.000000,912.000000,912.000000
mean,3.827255,98113.417466,7.640730,2007.845043,2011.116279,150.461612,47.625961,-122.331178,1.0,146.117509,7.685724e+04,2016.295728,4.378574e+09,1.238372,7.180233,129312.250000,122.232558,2009.040698,0.994318,0.954545,98113.495204,47.624680,-122.331544,99.994242,0.609405,6.992625e+04,1.733333,1.0,2.952767e+03,147.702851,7.419956,1.422149,0.017530,0.252217
std,0.513886,11.276347,6.469428,22.829139,9.366123,120.234860,0.040348,0.026369,0.0,132.438941,6.664557e+05,4.941117,2.993261e+09,0.848896,5.815593,154844.807924,120.089944,20.491435,0.075378,0.208893,11.457971,0.039219,0.026030,0.107211,0.488118,4.637888e+05,1.624221,0.0,9.530466e+04,114.867874,6.120287,0.857889,0.020756,0.177761
min,2.000000,98101.000000,1.000000,1900.000000,1978.000000,15.000000,47.520120,-122.388860,1.0,0.000000,0.000000e+00,2003.000000,7.600122e+06,1.000000,3.000000,1230.000000,3.000000,1916.000000,0.000000,0.000000,98101.000000,47.520092,-122.388821,98.000000,0.000000,1.000000e+00,1.000000,1.0,0.000000e+00,15.000000,1.000000,1.000000,0.001314,0.022727
25%,4.000000,98104.000000,5.000000,2009.000000,2008.000000,60.000000,47.610682,-122.347772,1.0,46.500000,1.500000e+03,2014.000000,1.986701e+09,1.000000,4.000000,32808.750000,45.000000,2012.000000,1.000000,1.000000,98104.000000,47.609846,-122.347362,100.000000,0.000000,1.000000e+00,1.000000,1.0,0.000000e+00,60.000000,5.000000,1.000000,0.006240,0.142857
50%,4.000000,98109.000000,6.000000,2015.000000,2015.000000,107.000000,47.618895,-122.328005,1.0,114.500000,5.659440e+03,2017.000000,3.213602e+09,1.000000,6.000000,67519.500000,74.000000,2016.000000,1.000000,1.000000,98109.000000,47.617976,-122.328548,100.000000,1.000000,1.000000e+00,1.000000,1.0,0.000000e+00,107.000000,6.000000,1.000000,0.011364,0.200000
75%,4.000000,98121.000000,7.000000,2019.000000,2016.500000,230.250000,47.656564,-122.315220,1.0,212.000000,1.987500e+04,2020.000000,7.457876e+09,1.000000,7.000000,160854.750000,152.500000,2018.000000,1.000000,1.000000,98121.000000,47.653106,-122.315948,100.000000,1.000000,2.250000e+00,1.500000,1.0,0.000000e+00,217.000000,7.000000,2.000000,0.020408,0.333333
max,5.000000,98199.000000,44.000000,2023.000000,2019.000000,761.000000,47.732554,-122.261650,1.0,916.000000,1.000000e+07,2025.000000,9.831200e+09,6.000000,41.000000,766545.000000,654.000000,2023.000000,1.000000,1.000000,98199.000000,47.732564,-122.261780,100.000000,1.000000,3.076434e+06,7.000000,1.0,3.076437e+06,761.000000,44.000000,6.000000,0.187500,1.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1042 entries, 0 to 1041
Data columns (total 83 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   addr_clean_x                     1042 non-null   object 
 1   BuildingName                     1031 non-null   object 
 2   StarRating                       1042 non-null   int64  
 3   Submarket                        1042 non-null   object 
 4   City                             1042 non-null   object 
 5   ZIPCode                          1042 non-null   int64  
 6   Stories                          1041 non-null   float64
 7   YearBuilt                        1039 non-null   float64
 8   YearRenov                        43 non-null     float64
 9   Class                            1042 non-null   object 
 10  TrueOwner                        1017 non-null   object 
 11  Units                            1042 non-null   float64
 12  latitude            

In [358]:
df_final_apt = df_final[df_final['Permit Class'].str.contains('Multifamily|Commercial', na=False)].copy()

df_final_apt.describe().T

,count,mean,std,min,25%,50%,75%,max
StarRating,986.0,3.835700e+00,5.151348e-01,2.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,5.000000e+00
ZIPCode,986.0,9.811351e+04,1.138787e+01,9.810100e+04,9.810400e+04,9.810900e+04,9.812100e+04,9.819900e+04
Stories,985.0,7.701523e+00,6.531525e+00,1.000000e+00,5.000000e+00,6.000000e+00,7.000000e+00,4.400000e+01
YearBuilt,983.0,2.007875e+03,2.313935e+01,1.900000e+03,2.009000e+03,2.015000e+03,2.019000e+03,2.023000e+03
YearRenov,41.0,2.010927e+03,9.556124e+00,1.978000e+03,2.006000e+03,2.015000e+03,2.017000e+03,2.019000e+03
Units,986.0,1.518398e+02,1.217579e+02,1.500000e+01,6.000000e+01,1.080000e+02,2.347500e+02,7.610000e+02
latitude,986.0,4.762617e+01,4.028560e-02,4.752012e+01,4.761068e+01,4.761879e+01,4.765656e+01,4.773255e+01
longitude,986.0,-1.223311e+02,2.609855e-02,-1.223889e+02,-1.223469e+02,-1.223274e+02,-1.223153e+02,-1.222617e+02
LEED,146.0,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
ParkingSpaces,516.0,1.454693e+02,1.324831e+02,0.000000e+00,4.400000e+01,1.155000e+02,2.120000e+02,9.160000e+02


## really start to narrow down for heatpump & for individual units
- if townhouse, n_building_ratio should >=1 --> installed for units
- if multifamily/mixed_use/other housing, 
    - if number of buildings == 1 & n>1, then n_unit_ratio > 4* 1/ number of floors --> installed for units
    - if n==1, check Number_final, if Number_final
- if mixed use|condo
    - if contains 'lounge|single zone|lobby' --> for common area

- if condo looks like townhouse, number of building >=3 & n_building_ratio >= 1 --> for all units


In [ ]:

# ###- if 'Detailed Existing Land Use' containts 'mixed use|condo' & - if 'Description' contains 'lounge|single zone|lobby' --> df_final['common_area_only'] = 1

# import numpy as np

# df_final1 = df_final_apt.copy()
# df_final1['common_area_only'] = np.nan
# # Ensure numeric columns are clean
# cols_to_fix = ['Number_final', 'Stories', 'Units']
# df_final1[cols_to_fix] = df_final1[cols_to_fix].apply(pd.to_numeric, errors='coerce')

# ##Note that when using np.select(conditions, choices), the first condition that evaluates to True for a given row is the one that gets applied. Later rules are ignored for that row once a match is found.

# # Define conditions
# conditions = [
#     # Rule 1: Common space keywords in shared housing --> for common areas
#     df_final1['Detailed Existing Land Use'].str.contains(
#         'Mixed-Use|Multi-Family|Other Housing|Condo', case=False, na=False
#     ) & df_final1['Description'].str.contains(
#         'lounge|single zone|lobby|retail', case=False, na=False
#     ),

#     # Rule 2: Townhouse or similar with per-unit ratio --> for units
#     df_final1['Detailed Existing Land Use'].str.contains(
#         'Townhouse', case=False, na=False
#     ) & (df_final1['n_building_ratio'] >= 1),

#     # Rule 3: Shared housing with high unit ratio --> for units
#     df_final1['Detailed Existing Land Use'].str.contains(
#         'Mixed-Use|Multi-Family|Other Housing|Condo', case=False, na=False) & #df_final1['common_area_only'].isna() &
#     (df_final1['n_unit_ratio'] > (4 / df_final1['Stories'])),

#     #  Rule 4: Shared housing with high system count --> for units
#     df_final1['Detailed Existing Land Use'].str.contains(
#         'Mixed-Use|Multi-Family|Other Housing|Condo', case=False, na=False
#     ) & #df_final1['common_area_only'].isna() &
#     (df_final1['Number_final'] > (df_final1['Units'] / 4)),

#     # Rule 5: Shared housing with very low unit ratio --> for common areas
#     df_final1['Detailed Existing Land Use'].str.contains(
#         'Mixed-Use|Multi-Family|Other Housing|Condo', case=False, na=False) & #df_final1['common_area_only'].isna() &
#     (df_final1['n_unit_ratio'] < (1 / df_final1['Stories'])),

#     # Rule 6: Condo looks like townhouse, number of building >=3 & n_building_ratio >= 1 --> for all units
#     df_final1['Detailed Existing Land Use'].str.contains(
#         'Condo', case=False, na=False) & (df_final1['n_building_ratio'] >= 1), #& #(df_final1['Number_final'] >= 3),

#     ## Rule 7: for single family house --> for units
#     df_final1['Detailed Existing Land Use'].str.contains(
#         'Single Family|Townhouse|Duplex/Triplex/4-Plex', case=False, na=False),

#     ## Rule 8: if 'Description' contains '(for|at|in|system) unit (followed by numbers or 'A-Z')' --> df_final['common_area_only'] = 0
#     df_final1['Description'].str.contains(r'(for|at|in|system)\s+unit\s*:?\s*(\w+|\d+)', case=False, na=False),

#     ## Rule 9: any cases that n_unit_ratio >=1 & not negative Number of Buildings --> for units
#     (df_final1['n_unit_ratio'] >= 1) & (df_final1['Number of Buildings_y'] >= 0)
# ]

# # Corresponding values
# choices = [1, 0, 0, 0, 1, 0, 0, 0, 0]

# # Apply CASE WHEN logic
# df_final1['common_area_only'] = np.select(conditions, choices, default=np.nan)



KeyError: 'n_building_ratio'

In [359]:
### write the df_final1 to excel for checking
df_final2 = df_final #[df_final1['HP_related'] == 1 & df_final1['Detailed Existing Land Use'].str.contains(
       # 'Mixed-Use|Multi-Family|Other Housing|Condo|Townhouse', case=False, na=False)]
df_final2.describe().T

,count,mean,std,min,25%,50%,75%,max
StarRating,1042.0,3.827255e+00,5.138859e-01,2.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,5.000000e+00
ZIPCode,1042.0,9.811342e+04,1.127635e+01,9.810100e+04,9.810400e+04,9.810900e+04,9.812100e+04,9.819900e+04
Stories,1041.0,7.640730e+00,6.469428e+00,1.000000e+00,5.000000e+00,6.000000e+00,7.000000e+00,4.400000e+01
YearBuilt,1039.0,2.007845e+03,2.282914e+01,1.900000e+03,2.009000e+03,2.015000e+03,2.019000e+03,2.023000e+03
YearRenov,43.0,2.011116e+03,9.366123e+00,1.978000e+03,2.008000e+03,2.015000e+03,2.016500e+03,2.019000e+03
Units,1042.0,1.504616e+02,1.202349e+02,1.500000e+01,6.000000e+01,1.070000e+02,2.302500e+02,7.610000e+02
latitude,1042.0,4.762596e+01,4.034817e-02,4.752012e+01,4.761068e+01,4.761889e+01,4.765656e+01,4.773255e+01
longitude,1042.0,-1.223312e+02,2.636898e-02,-1.223889e+02,-1.223478e+02,-1.223280e+02,-1.223152e+02,-1.222617e+02
LEED,157.0,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
ParkingSpaces,546.0,1.461175e+02,1.324389e+02,0.000000e+00,4.650000e+01,1.145000e+02,2.120000e+02,9.160000e+02


## Get permit content for each


In [262]:
import requests
from bs4 import BeautifulSoup
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Optional selenium fallback (only used if requests/BS can't get full content)
try:
    _SELENIUM_AVAILABLE = True
except Exception:
    _SELENIUM_AVAILABLE = False

def _get_visible_text_from_html(html):
    soup = BeautifulSoup(html, "html.parser")
    # remove script/style
    for s in soup(["script", "style", "noscript"]):
        s.decompose()
    text = soup.get_text(separator="\n", strip=True)
    # collapse whitespace
    return "\n".join([line.strip() for line in text.splitlines() if line.strip()])

def scrape_permit_content(
    df,
    link_col="Link",
    content_col="Permit_Content",
    min_delay=1.0,
    max_delay=3.0,
    requests_timeout=20,
    user_agent=None,
    selenium_fallback=True,
    selenium_headless=True
):
    """
    Follow each URL in df[link_col], fetch the page, extract visible text and store in df[content_col].
    Respects polite delays between requests to avoid triggering site protection.
    If requests + BeautifulSoup returns a very small text, optionally fall back to Selenium (if available).
    Returns the modified dataframe (in-place update as well).
    """
    headers = {
        "User-Agent": user_agent or "Mozilla/5.0 (compatible; DataScraper/1.0; +https://example.org/bot)"
    }

    # Prepare selenium driver only if needed
    driver = None
    if selenium_fallback and _SELENIUM_AVAILABLE:
        chrome_opts = Options()
        if selenium_headless:
            chrome_opts.add_argument("--headless=new")
        chrome_opts.add_argument("--disable-gpu")
        chrome_opts.add_argument("--no-sandbox")
        chrome_opts.add_argument("--disable-dev-shm-usage")
        # Prevent webdriver logs from printing too much
        chrome_opts.add_argument("--log-level=3")
        # don't create driver yet; create lazily if fallback is required
    else:
        selenium_fallback = False

    contents = []
    total = len(df)
    for idx, url in enumerate(df[link_col].fillna("").astype(str).values):
        if not url or url.strip().lower() in ("nan", "none", ""):
            contents.append("")
            continue

        try:
            # polite random sleep
            time.sleep(random.uniform(min_delay, max_delay))

            resp = requests.get(url, headers=headers, timeout=requests_timeout)
            if resp.status_code == 200 and resp.content:
                text = _get_visible_text_from_html(resp.content)
            else:
                text = ""
        except Exception as e:
            text = ""

        # If content looks too small and selenium fallback allowed, try selenium (some pages require JS)
        if (not text or len(text) < 200) and selenium_fallback:
            try:
                if driver is None:
                    driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_opts)
                driver.get(url)
                # wait for body to load (adjust timeout as needed)
                try:
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
                except Exception:
                    pass
                page_source = driver.page_source
                text = _get_visible_text_from_html(page_source)
            except Exception:
                # swallow selenium errors and keep whatever text we have (maybe empty)
                pass

        contents.append(text)

    # close selenium driver if opened
    try:
        if driver:
            driver.quit()
    except Exception:
        pass

    # attach results to dataframe
    df[content_col] = contents
    return df

# Example usage on the existing dataframe 'df_final2' (the notebook already defines this variable)
# df_final2 = scrape_permit_content(df_final2, link_col='Link', content_col='Permit_Content', min_delay=1.5, max_delay=4.0)
# Optionally save progress: df_final2.to_pickle('./02ProcessedData/df_final2_with_permit_content.pkl')

#### The following code will take about 80 mins to run -- do it unless necessary

In [ ]:
# df_final2 = scrape_permit_content(df_final2, link_col='Link', content_col='Permit_Content', min_delay=0.5, max_delay= 2)


In [ ]:
# df_final2.to_excel('02ProcessedData/merged_apt_trade_HP_permit_content.xlsx', index=False)

In [471]:
df.head()
df.info()

,Number,Quarter,Status,AskingRentPerUnit,AskingRentPerSF,AskingRentGrowthYr,EffectiveRentPerUnit,EffectiveRentPerSF,EffectiveRentGrowthYr,VacancyUnits,VacancyTotal,Address,BuildingName,StarRating,ES,Esyearscertified,Eslast,LEED,GreenRating,LEEDtype,Points,version,RBAGLA,Submarket,City,ZIPCode,Stories,YearBuilt,YearRenov,Class,TrueOwner,Units,Style,AvgConcessions,StudiosUnits,Onebedunits,Twobedunits,Threebedunits,Fourbedunits,RentType,AffordableType,ParkingSpacesUnit,ParkingSpaces,MarketSegment,latitude,longitude,Neighborhood,addr_clean,has heat pump,addr_clean_x,coverage_classification_final,Completed Date,HP_related,Year Completed,HP_covered_residential_units,Quarter_clean,Quarter_period,Quarter_end,Completed_ts,is_before,is_after,is_same_or_after,HP_installed
0,MBSEA272,2010 Q4,Existing,1245.6364,1.610721,0.042711,1242.4318,1.606577,0.042826,0.0000,0.000000,3800 Aurora Ave N,Northbridge Apartments,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39063,Fremont,Seattle,98103,4.0,1994.0,NaN,B,SSS Properties,44.0,Mid-Rise,0.3,NaN,55.4,21.4,23.2,NaN,Market,NaN,1.27,56.0,All,47.653136,-122.34682,FREMONT,3800 AURORA AVE N,No,NaN,NaN,NaN,NaN,NaN,NaN,2010Q4,2010Q4,2010-12-31,NaT,False,False,False,0
1,MBSEA366,2000 Q1,Existing,NaN,NaN,NaN,NaN,NaN,NaN,1.1695,0.041768,4821 Fauntleroy Way SW,Thunderbird Apartments,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22400,Fairmount Park,Seattle,98116,2.0,1957.0,NaN,B,1: Edwin N Spaunhurst | 2: John N Garner | 3: ...,28.0,Low-Rise,0.0,NaN,89.3,10.7,NaN,NaN,Market,NaN,NaN,0.0,All,47.558564,-122.38193,ALASKA JUNCTION,4821 FAUNTLEROY WAY SW,No,NaN,NaN,NaN,NaN,NaN,NaN,2000Q1,2000Q1,2000-03-31,NaT,False,False,False,0
2,MBSEA536,2001 Q4,Existing,1324.5068,1.324507,0.054906,1311.2534,1.311253,0.054904,19.1916,0.131449,10735 Roosevelt Way NE,Park at Northgate Apartment Homes,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,146000,Maple Leaf,Seattle,98125,3.0,1967.0,1995.0,B,"Wallace Properties, Inc.",146.0,Garden,0.0,NaN,NaN,100.0,NaN,NaN,Market,NaN,1.64,239.0,All,47.707318,-122.31922,NORTHGATE,10735 ROOSEVELT WAY NE,No,NaN,NaN,NaN,NaN,NaN,NaN,2001Q4,2001Q4,2001-12-31,NaT,False,False,False,0
3,MBSEA353,2011 Q3,Existing,1527.6866,1.974225,0.068602,1516.6468,1.959958,0.067336,12.3010,0.061199,1211-1215 Dexter Ave N,Dexter Lake Union,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,286408,Westlake,Seattle,98109,11.0,2004.0,NaN,B,The Sobrato Organization,201.0,Mid-Rise,0.9,18.9,54.7,26.4,NaN,NaN,Market,NaN,0.27,54.0,All,47.630066,-122.34263,QUEEN ANNE,12111215 DEXTER AVE N,No,NaN,NaN,NaN,NaN,NaN,NaN,2011Q3,2011Q3,2011-09-30,NaT,False,False,False,0
4,MBSEA197,2023 Q4 QTD,Existing,NaN,NaN,NaN,NaN,NaN,NaN,2.6271,0.050521,1307 N 43rd St,Pickering Place Apartments,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8000,Wallingford,Seattle,98103,5.0,NaN,NaN,B,Burton Walls Investments,52.0,Low-Rise,0.0,NaN,100.0,NaN,NaN,NaN,Market,NaN,NaN,NaN,All,47.658971,-122.34192,WALLINGFORD,1307 N 43RD ST,No,NaN,NaN,NaN,NaN,NaN,NaN,2023Q4,2023Q4,2023-12-31,NaT,False,False,False,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65436 entries, 0 to 65435
Data columns (total 63 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Number                         65436 non-null  object        
 1   Quarter                        65436 non-null  object        
 2   Status                         65436 non-null  object        
 3   AskingRentPerUnit              57690 non-null  float64       
 4   AskingRentPerSF                55906 non-null  float64       
 5   AskingRentGrowthYr             56668 non-null  float64       
 6   EffectiveRentPerUnit           57690 non-null  float64       
 7   EffectiveRentPerSF             55906 non-null  float64       
 8   EffectiveRentGrowthYr          56668 non-null  float64       
 9   VacancyUnits                   65359 non-null  float64       
 10  VacancyTotal                   65359 non-null  float64       
 11  Address        

In [472]:
# in Permit_Content, remove extra whitespace and line breaks. Also remove any content before 'Record Number' 
# or remove any content including and after 'You can find all fees assessed'
df_final2 = pd.read_excel('02ProcessedData/merged_apt_trade_HP_permit_content.xlsx')

##drop most of df_final2 columns except 'Permit Number' and 'Permit_Content' and then merge with df_final1
df_final2 = df_final2[['Permit Number', 'Permit_Content']].merge(
    df_final,
    on='Permit Number', #,'Year Completed'],
    how='left',
    suffixes=('', '_y')
)

df_final2['Permit_Content1'] = df_final2['Permit_Content'].str.replace(r'\s+', ' ', regex=True).str.strip()
df_final2['Permit_Content1'] = df_final2['Permit_Content1'].str.split('Record Number').str[-1]
df_final2['Permit_Content1'] = df_final2['Permit_Content1'].str.split('You can find all fees assessed').str[0]
df_final2['Permit_Content'] = df_final2['Permit_Content1'].str.strip()
df_final2['Permit_Content'].head()
df_final2.to_excel('02ProcessedData/check_pertmit_for_common_area.xlsx', index=False)
# df_final2['Year Completed'].value_counts()

0    : 6360063-RF : Refrigeration Permit Record Sta...
1    : 6671391-RF : Refrigeration Permit Record Sta...
2    : 6091968-RF : Refrigeration Permit Record Sta...
3    : 6175558-RF : Refrigeration Permit Record Sta...
4    : 6231772-RF : Refrigeration Permit Record Sta...
Name: Permit_Content, dtype: object

In [474]:


## find unique permit number for df_final2 -- make the data distinct on 'Permit Number'
df_final2 = df_final2.drop_duplicates(subset=['Permit Number'])
df_final2.describe(include='object').T

,count,unique,top,freq
Permit Number,934,934,6360063-RF,1
Permit_Content,934,934,: 6360063-RF : Refrigeration Permit Record Sta...,1
addr_clean_x,934,493,900 university st,15
BuildingName,923,485,Horizon House,15
Submarket,934,62,Capitol Hill,123
City,934,1,Seattle,934
Class,934,2,A,570
TrueOwner,910,313,Equity Residential,28
Neighborhood,934,45,CAPITOL HILL,125
GreenRating,141,8,Silver,49


## GPT suggested data cleaning for the permit content
The idea is to extract the tonnage data and compressor data
In general, 1 tonnage of compressors work for 400 -600 sqft of space.

In [325]:
import re
import pandas as pd
from fractions import Fraction
from collections import Counter

WORD2NUM = {
    "one":1,"two":2,"three":3,"four":4,"five":5,"six":6,"seven":7,"eight":8,"nine":9,"ten":10,
    "eleven":11,"twelve":12,"thirteen":13,"fourteen":14,"fifteen":15,"sixteen":16,
    "seventeen":17,"eighteen":18,"nineteen":19,"twenty":20
}


# 1) ONE final PERMIT_RE. Newline-agnostic end boundary; '*' sections optional.
PERMIT_RE = re.compile(
    r":\s*(\d{7}-RF)\s*:\s*Refrigeration Permit"                # id + header
    r"(?:.*?\*\s*.*?\*\s*)?"                                    # optional star sections
    r"(?P<body>.*?)"                                            # details body
    r"(?=(?:\s*:\s*\d{7}-RF\s*:\s*Refrigeration Permit)|\Z)",   # next start OR end
    re.IGNORECASE | re.DOTALL
)

def _first(pat, s):
    m = re.search(pat, s, re.I|re.S)
    return m.group(1).strip() if m else None

def _first_int(pat, s):
    v = _first(pat, s)
    return int(v) if v and v.isdigit() else None

def _to_float_tons(x: str) -> float:
    x = x.strip().lower()
    if re.match(r"^\d+\s+\d+/\d+$", x):
        m = re.match(r"^(\d+)\s+(\d+)/(\d+)$", x)
        return int(m.group(1)) + Fraction(int(m.group(2)), int(m.group(3)))
    if re.match(r"^\d+/\d+$", x):
        num, den = x.split("/")
        return Fraction(int(num), int(den))
    return float(x)

def _normalize(s: str) -> str:
    # keep newlines; only collapse spaces/tabs
    s = s.replace("\u00bd","1/2").replace("\u00bc","1/4").replace("\u00be","3/4")
    s = re.sub(r"[–—]", "-", s)
    s = re.sub(r"[ \t]+", " ", s)
    return s

def parse_permits_full(raw_text: str) -> pd.DataFrame:
    text = _normalize(raw_text)
    rows = []

    for m in PERMIT_RE.finditer(text):
        permit_id = m.group(1)
        chunk     = m.group(0)
        body      = m.group('body') or chunk   # <-- use named group

        status   = _first(r"Record Status:\s*([A-Za-z ]+)", chunk)  # <-- fixed
        address  = _first(r"Work Location\s+([^\n\*]+)", chunk)
        declared_total = _first_int(r"Number of Compressors:\s*(\d+)", chunk)

        used_spans, contrib = [], []
        def mark_span(sp): used_spans.append(sp)
        def span_free(sp):
            a,b = sp
            for x,y in used_spans:
                if not (b <= x or a >= y): return False
            return True

        # A) Tonnage + explicit count
        for mm in re.finditer(r"Tonnage:\s*([0-9]+(?:\.[0-9]+)?)\s+Number of Compressors at this Tonnage:\s*(\d+)", body, re.I):
            if span_free(mm.span()):
                t = float(mm.group(1)); c = int(mm.group(2))
                contrib.append((t*c, c, "A")); mark_span(mm.span())

        # B) Pair "Tonnage:" with "Number of Compressors on Chiller:"
        for bb in re.finditer(r"(REFRIGERATION SYSTEM|COMPRESSORS).*?(?=(?:REFRIGERATION SYSTEM|COMPRESSORS|\Z))", body, re.I|re.S):
            block = bb.group(0); bl, br = bb.span()
            tons = [(m2.group(0),(m2.start()+bl,m2.end()+bl),float(m2.group(1)))
                    for m2 in re.finditer(r"Tonnage:\s*([0-9]+(?:\.[0-9]+)?)", block, re.I)]
            chs  = [(m3.group(0),(m3.start()+bl,m3.end()+bl),int(m3.group(1)))
                    for m3 in re.finditer(r"Number of Compressors on Chiller:\s*(\d+)", block, re.I)]
            if not tons or not chs: 
                pass
            elif len(chs)==len(tons):
                for (_,sp_t,t),(_,sp_c,c) in zip(tons, chs):
                    span=(min(sp_t[0],sp_c[0]), max(sp_t[1],sp_c[1]))
                    if span_free(span): contrib.append((t*c,c,"B")); mark_span(span)
            elif len(chs)==1:
                c = chs[0][2]
                for _,sp_t,t in tons:
                    if span_free(sp_t): contrib.append((t*c,c,"B*")); mark_span(sp_t)

        # C) n @ x tons
        for mm in re.finditer(r"(\d+)\s*@\s*([0-9]+(?:\.[0-9]+)?)\s*tons?", body, re.I):
            if span_free(mm.span()):
                c=int(mm.group(1)); t=float(mm.group(2))
                contrib.append((t*c,c,"@")); mark_span(mm.span())

        # D) (n) x-ton (incl. fractions)
        for mm in re.finditer(r"\(\s*(\d+)\s*\)\s*([0-9]+(?:\.[0-9]+)?|\d+\s+\d+/\d+|\d+/\d+)\s*[- ]?tons?", body, re.I):
            if span_free(mm.span()):
                c=int(mm.group(1)); t=float(_to_float_tons(mm.group(2)))
                contrib.append((t*c,c,"D")); mark_span(mm.span())

        # E) BTU w/ count
        WORD2NUM = {"one":1,"two":2,"three":3,"four":4,"five":5,"six":6,"seven":7,"eight":8,"nine":9,"ten":10,
                    "eleven":11,"twelve":12,"thirteen":13,"fourteen":14,"fifteen":15,"sixteen":16,"seventeen":17,
                    "eighteen":18,"nineteen":19,"twenty":20}
        for mm in re.finditer(r"\b(one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|\d+)\b\s*([0-9][0-9,]{2,})\s*btu", body, re.I):
            if span_free(mm.span()):
                tok=mm.group(1).lower()
                # c=WORD2NUM.get(tok, int(tok))
                if tok.isdigit():
                    c = int(tok)
                else:
                    c = WORD2NUM.get(tok, 1)  # sensible fallback if a rare word slips through
                btu=int(mm.group(2).replace(",",""))
                t=btu/12000.0
                contrib.append((t*c,c,"BTU")); mark_span(mm.span())
        # E2) lone BTU → one unit
        for mm in re.finditer(r"\b([0-9][0-9,]{2,})\s*btu", body, re.I):
            if span_free(mm.span()):
                btu=int(mm.group(1).replace(",",""))
                t=btu/12000.0
                contrib.append((t*1,1,"BTU1")); mark_span(mm.span())

        # F) lone "Tonnage:"
        for mm in re.finditer(r"Tonnage:\s*([0-9]+(?:\.[0-9]+)?)", body, re.I):
            if span_free(mm.span()):
                t=float(mm.group(1))
                contrib.append((t*1,1,"F")); mark_span(mm.span())

        # G: Match things like: "Two 6-25 HP compressors", "(3) 000-005HP", "4 25 HP", etc.
        # Blocks to avoid cross-pairing
        RE_G_BLOCKS = re.compile(
            r"(?:Compressor Type|Type of Compressor|REFRIGERATION SYSTEM|COMPRESSORS).*?"
            r"(?=(?:Compressor Type|Type of Compressor|REFRIGERATION SYSTEM|COMPRESSORS|\Z))",
            re.I | re.S
        )
        # Tokenizer: only two named groups we care about: count and label
        RE_G_TOKEN = re.compile(
            r"""
            (?:
                Number\s+of\s+Compressors(?:\s+on\s+Chiller)?\s*:\s*(?P<count>\d+)
            |
                Compressor\s+Type\s*:\s*(?P<label>\d{3}(?:-\d{3})?)\s*H\.?\s*P\.?
            )
            """,
            re.I | re.X
        )

        type_counts = Counter()

        # def _pair_count_types(block: str, base: int):
        #     pending_count = None
        #     for m in RE_G_TOKEN.finditer(block):
        #         sp = (base + m.start(), base + m.end())
        #         if not span_free(sp):
        #             continue

        #         if m.group("count") is not None:
        #             pending_count = int(m.group("count"))
        #             mark_span(sp)

        #         elif m.group("label") is not None:
        #             c = pending_count if pending_count is not None else 1
        #             label_norm = f"{m.group('label')} HP"
        #             type_counts[label_norm] += c
        #             contrib.append((0, c, "GPAIR"))
        #             mark_span(sp)
        #             pending_count = None  # consume the count

        # # Prefer blocks; else scan whole body
        # blocks = list(RE_G_BLOCKS.finditer(body))
        # if blocks:
        #     for b in blocks:
        #         _pair_count_types(b.group(0), b.start())
        # else:
        #     _pair_count_types(body, 0)

        
        def parse_types_with_pairing(text: str) -> Counter:
            """
            Read-only: does NOT use span_free/mark_span and does NOT touch contrib.
            Pairs the most recent 'count' with the next 'label' in sequence. If a label
            appears with no prior count, defaults to 1.
            """
            type_counts = Counter()

            def _scan(block: str):
                pending = None
                for m in RE_G_TOKEN.finditer(block):
                    if m.group("count") is not None:
                        pending = int(m.group("count"))
                    elif m.group("label") is not None:
                        c = pending if pending is not None else 1
                        type_counts[f"{m.group('label')} HP"] += c
                        pending = None

            blocks = list(RE_G_BLOCKS.finditer(text))
            if blocks:
                for b in blocks:
                    _scan(b.group(0))
            else:
                _scan(text)

            return type_counts

        type_counts = parse_types_with_pairing(body)


        compressors_detailed = sum(c for _,c,_ in contrib)
        tonnage_total = round(sum(t for t,_,_ in contrib), 3)

        notes = [tag for *_,tag in contrib]
        if compressors_detailed==0 and declared_total:
            loose = [float(x.group(1)) for x in re.finditer(r"Tonnage:\s*([0-9]+(?:\.[0-9]+)?)", body, re.I)]
            if len(loose)==declared_total:
                tonnage_total = round(sum(loose), 3)
                compressors_final = declared_total
                notes.append("C_reconcile")
            else:
                compressors_final = declared_total
                notes.append(f"C_declared_only:{declared_total}")
        else:
            compressors_final = compressors_detailed or (declared_total or 0)

        # rows.append({
        #     "permit_id": permit_id,
        #     # "status": status,
        #     # "address": address,
        #     "compressors_types_found": sum(1 for _,_,tag in contrib if tag=="G"),
        #     "compressors_declared": declared_total,
        #     "compressors_detailed": compressors_detailed or None,
        #     "compressors_final": compressors_final,
        #     "tonnage_total": tonnage_total #,
        #    # "notes": ",".join(notes) if notes else None
        # })
        # rows.append({
        #     "permit_id": permit_id,
        #     "compressors_types_found": sum(type_counts.values()),  # total compressor count from text
        #     "compressor_type_counts": dict(type_counts) if type_counts else None,
        #     "compressors_declared": declared_total,
        #     "compressors_detailed": compressors_detailed or None,
        #     "compressors_final": compressors_final,
        #     "tonnage_total": tonnage_total
        # })
        # Save per-permit outputs
        rows.append({
            "permit_id": permit_id,
            "compressors_types_found": sum(type_counts.values()),      # total actual compressors from G pairing
            "compressor_type_counts": dict(type_counts) or None,       # {"000-005 HP": 2, "006-025 HP": 1}
            "compressors_declared": declared_total,
            "compressors_detailed": compressors_detailed or None,
            "compressors_final": compressors_final,
            "tonnage_total": tonnage_total
        })

    return pd.DataFrame(rows).sort_values("permit_id").reset_index(drop=True)


In [475]:
# raw_text = df_final2.loc[:, 'Permit_Content'].to_string(index=False)
# raw_text = "\n".join(df_final2["Permit_Content"].dropna().astype(str))
# Good: preserve record separators and tame spaces/tabs
# Build raw_text (keep newlines, trim spaces/tabs)
raw_text = "\n".join(df_final2["Permit_Content"].dropna().astype(str))
raw_text = re.sub(r"[ \t]+", " ", raw_text)

# Quick sanity: how many headers are present?
hdrs = re.findall(r":\s*\d{7}-RF\s*:\s*Refrigeration Permit", raw_text, flags=re.I)
print("Header count:", len(hdrs))


print(raw_text[1:5000])  # print first 5000 characters for checking
# raw_text = ": 6695017-RF : Refrigeration Permit Record Status: Completed Expiration Date: 04/22/2020 Create a New Collection * Name: Description: Add Cancel Record Info Record Details Status Related Records Attachments Inspections & Appointments Payments Fees Conditions Work Location 3004 S ALASKA ST 161368 RPZ ELIGIBLE: Y TYPE OF RESIDENCE: Residence LOW INCOME: N RPZ ID: 29 RPZ DESCRIPTION: COLUMBIA CITY RPZ LIMIT - DECAL: 4 RPZ LIMIT - GUEST: 1 RPZ LIMIT - ONE DAY BY YEAR: 50 RPZ ISSUED - DECAL: 0 RPZ ISSUED - GUEST: 0 RPZ ISSUED - SHORT-TERM: 0 RPZ LIMIT - BUSINESS DECAL: 0 RPZ LIMIT - BUSINESS GUEST: 0 RPZ LIMIT - SHORT-TERM: 5 RPZ ISSUED - BUSINESS DECAL: 0 RPZ ISSUED - BUSINESS GUEST: 0 100 BLOCK: 3000 BLOCK S ALASKA ST * Record Details Project Description Sonata East Install refrigeration piping for 5 split systems. More Details Application Information PERMIT APPLICATION COMMON Where on your property are you working?: Parking level, 1st floor and roof. Is work in the right-of-way (street or alley) required?: No Choose the Primary Property Use: Multifamily Project Value: 622 PERMIT TRACKING COMMON Review Level: Field Application Completed Date: 10/22/2018 Permit Issued Date: 10/22/2018 Inspections Completed Date: 12/28/2018 EQUIPMENT INFORMATION Type of Work: New Number of Compressors: 5 Other Information REFRIGERATION SYSTEM Equipment Type: Stand Alone Compressor Manufacturer: Mitsibushi Refrigerant Type and Class: R-410A(A1) Tonnage: 3.5 Horsepower: 1 Number of Safety Valves: 0 Safety Valve Discharge Location: none Life Safety System: No Number of Compressors on Chiller: 1 Compressor Type: Twin Rotary Equipment Type: Stand Alone Compressor Manufacturer: Mitsibushi Refrigerant Type and Class: R-410A(A1) Tonnage: 2.5 Horsepower: 0.5 Number of Safety Valves: 0 Safety Valve Discharge Location: none Life Safety System: No Number of Compressors on Chiller: 1 Compressor Type: Twin Rotary Equipment Type: Stand Alone Compressor Manufacturer: Mitsibushi Refrigerant Type and Class: R-410A(A1) Tonnage: 0.75 Horsepower: 0.25 Number of Safety Valves: 0 Safety Valve Discharge Location: none Life Safety System: No Number of Compressors on Chiller: 1 Compressor Type: Twin Rotary Equipment Type: Stand Alone Compressor Manufacturer: Mitsibushi Refrigerant Type and Class: R-410A(A1) Tonnage: 1 Horsepower: 0.25 Number of Safety Valves: 0 Safety Valve Discharge Location: none Life Safety System: No Number of Compressors on Chiller: 1 Compressor Type: Twin Rotary Equipment Type: Stand Alone Compressor Manufacturer: Mitsibushi Refrigerant Type and Class: R-410A(A1) Tonnage: 0.75 Horsepower: 0.25 Number of Safety Valves: 0 Safety Valve Discharge Location: none Life Safety System: No Number of Compressors on Chiller: 1 Compressor Type: Twin Rotary Parcel Information No parcel information Fees"
# raw_text = ': 6973044-RF : Refrigeration Permit Record Status: Completed Expiration Date: 12/29/2024 Create a New Collection * Name: Description: Add Cancel Record Info Record Details Status Related Records Attachments Inspections & Appointments Payments Fees Conditions Work Location 8022 15TH AVE NW 56913 STR ADDRESS ELIGIBILITY: No Exception * Record Details Project Description Noba Heat pump installation More Details Application Information PERMIT APPLICATION COMMON Is work in the right-of-way (street or alley) required?: No Choose the Primary Property Use: Multifamily Project Value: 800 PERMIT TRACKING COMMON Review Level: Field Application Completed Date: 06/29/2023 Permit Issued Date: 06/29/2023 Inspections Completed Date: 08/31/2023 FLOATING RESIDENCE INFO Is this building a Floating Residence?: No EQUIPMENT INFORMATION Type of Work: Addition/Alteration Other Information COMPRESSORS Tonnage: 1 Number of Compressors at this Tonnage: 1 Parcel Information Development Site Parcel:DV0017204 * Block:-- Lot:-- Subdivision:-- Fees'
# df_permit_parsed =parse_permits(raw_text) # parse_permits_full(raw_text)
df_permit_parsed = parse_permits_full(raw_text)

import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

print(df_permit_parsed.tail())



df_permit_parsed[df_permit_parsed['compressors_types_found']>0]['compressors_types_found'].value_counts()
df_permit_parsed.to_excel('02ProcessedData/parsed_permit_details_new_method.xlsx', index=False)

Header count: 934
 6360063-RF : Refrigeration Permit Record Status: Completed Expiration Date: 10/05/2014 Create a New Collection * Name: Description: Add Cancel Record Info Record Details Status Related Records Attachments Inspections & Appointments Payments Fees Conditions Work Location 2428 NW MARKET ST 72884 * Record Details Project Description REFRIGERATION PIPING FOR 8 SPLIT SYSTEM HEAT PUMPS More Details Application Information PERMIT APPLICATION COMMON Choose the Primary Property Use: Commercial PERMIT TRACKING COMMON Review Level: Field Application Completed Date: 04/05/2013 Permit Issued Date: 04/05/2013 Inspections Completed Date: 08/13/2014 EQUIPMENT INFORMATION Type of Work: New Number of Temporary Installations: 0 Number of Compressors: 8 Other Information REFRIGERATION SYSTEM Number of Compressors on Chiller: 8 Compressor Type: 000-005 HP Parcel Information Development Site Parcel:DV0078073 * Block:-- Lot:-- Subdivision:-- Fees
: 6671391-RF : Refrigeration Permit Record 

compressors_types_found
1    511
2     29
3      3
Name: count, dtype: int64

## Redo the Classification of the Sharable Space

parameters: sqft/ton
multi-family -- -550 -700
high-performance -- 700 -900

common area ratio:
garden, low rise: 10% -15%
mid-rise: 15% -25%
high-rise, mixed use: 25% -35%

In [476]:
# df_final2 = pd.read_excel('02ProcessedData/check_pertmit_for_common_area.xlsx')

import re
from typing import List

# Matches: "000-005 HP", "006-025HP", "123HP", case-insensitive, allows "H.P."
TYPE_RE = re.compile(r'\b(\d{3}(?:-\d{3})?)\s*H\.?\s*P\.?\b', re.I)

def extract_hp_types(text: str, unique: bool = False) -> List[str]:
    """
    Return a list of compressor type labels normalized to 'NNN[-NNN]HP'.
    If unique=True, deduplicate while preserving order.
    """
    labels = [f"{m.group(1).upper()}HP" for m in TYPE_RE.finditer(text or "")]
    if unique:
        labels = list(dict.fromkeys(labels))  # order-preserving dedupe
    return labels
# If your DataFrame has the full permit text in column 'raw_text'
df_final2['compressor_type_raw'] = df_final2['Permit_Content'].apply(lambda s: ", ".join(extract_hp_types(s, unique=True)) or None)

df_final2['compressor_type_raw'].value_counts()

## check if the system is VRF system
df_final2['is_VRF_system'] = df_final2['Permit_Content'].str.contains(
    r'\bVRF\b|\bVariable Refrigerant Flow\b', case=False, na=False
) | df_final2['Description'].str.contains(
    r'\bVRF\b|\bVariable Refrigerant Flow\b', case=False, na=False
)


## check if the system is split system
df_final2['is_split_system'] = df_final2['Permit_Content'].str.contains(
    r'\bsplit\b|\bsplit-system\b|\bsplit system\b|\bmini-split\b|\bmini split\b', case=False, na=False
) | df_final2['Description'].str.contains(
    r'\bsplit\b|\bsplit-system\b|\bsplit system\b|\bmini-split\b|\bmini split\b', case=False, na=False
)

## check if it is fan coil system
df_final2['is_fan_coil_system'] = df_final2['Permit_Content'].str.contains(
    r'\bfan coil\b|\bfancoil\b', case=False, na=False
) | df_final2['Description'].str.contains(
    r'\bfan coil\b|\bfancoil\b', case=False, na=False
)

### make a table for the permit number, completed year and whether it is VRF system, split system or fan coil system; 
# if later permit from same address didn't address that, still consider it a VRF system/split system/fan coil system
df_systems = df_final2.groupby(['addr_clean_x', 'Year Completed']).agg(
    is_VRF_system=('is_VRF_system', 'max'),
    is_split_system=('is_split_system', 'max'),
    is_fan_coil_system=('is_fan_coil_system', 'max')
).reset_index()
## add columns 'VRF_year;, 'split_year' and 'fan_coil_year' for those are TRue
# df_systems['VRF_year'] = df_systems.apply(
#     lambda row: row['Year Completed'] if row['is_VRF_system'] else None, axis=1
# )
# df_systems['split_year'] = df_systems.apply(
#     lambda row: row['Year Completed'] if row['is_split_system'] else None, axis=1
# )
# df_systems['fan_coil_year'] = df_systems.apply(
#     lambda row: row['Year Completed'] if row['is_fan_coil_system'] else None, axis=1
# )

# ## for df_systems, change Is_VRF_system, is_split_system, is_fan_coil_system to True if any year is later than 'VRF_year', 'split_year', 'fan_coil_year' respectively
# df_systems['is_VRF_system'] = df_systems.apply(
#     lambda row: True if row['Year Completed'] >= row['VRF_year'] and row['VRF_year'] is not None else False, axis=1
# )
# df_systems['is_split_system'] = df_systems.apply(  
#     lambda row: True if row['Year Completed'] >= row['split_year'] and row['split_year'] is not None else False, axis=1
# )
# df_systems['is_fan_coil_system'] = df_systems.apply(
#     lambda row: True if row['Year Completed'] >= row['fan_coil_year'] and row['fan_coil_year'] is not None else False, axis=1
# )   

# df_systems.info()
# df_final2 = df_final2.drop(columns=['is_VRF_system', 'is_split_system', 'is_fan_coil_system'])
# df_final2 = df_final2.merge(df_systems, on='addr_clean_x', how='left')

df_final2['is_fan_coil_system'].value_counts()

df_final2['is_split_system'].value_counts()

df_final2['is_VRF_system'].value_counts()

df_final2[df_final2['is_VRF_system'] == True].head()

df_final2[df_final2['is_split_system'] == True].head()

df_final2.to_excel('02ProcessedData/apt_trade_HP_permit_details_1028.xlsx', index=False)


compressor_type_raw
000-005HP                          487
006-025HP                           21
000-005HP, 006-025HP                19
006-025HP, 000-005HP                 9
026-100HP                            3
000-005HP, 026-100HP, 006-025HP      2
006-025HP, 026-100HP                 1
006-025HP, 000-005HP, 026-100HP      1
Name: count, dtype: int64

is_fan_coil_system
False    921
True      13
Name: count, dtype: int64

is_split_system
False    525
True     409
Name: count, dtype: int64

is_VRF_system
False    897
True      37
Name: count, dtype: int64

,Permit Number,Permit_Content,addr_clean_x,BuildingName,StarRating,Submarket,City,ZIPCode,Stories,YearBuilt,YearRenov,Class,TrueOwner,Units,latitude,longitude,Neighborhood,LEED,GreenRating,ParkingSpaces,Style,Permit Class,Permit Class Mapped,Perm,Permit Type Mapped,Description,Appliance Type,Est Project Cost,Contractor Name,Applied Date,Issued Date,Expires Date,Completed Date,Month Applied For,Year Completed,Current Status,PIN,Address,Property Name,Detailed Existing Land Use,Planning Existing Land Use Code,General Existing Land Use,Ownership Type,Number of Buildings,Number of Floors,20K,Existing Building Square Feet,Number of Units,Year Built Group,Year Built,Neighborhoods,Link,Issued,Completed,addr_clean_y,PermitTypeMapped,IssuedDate,StatusCurrent,OriginalCity,OriginalState,OriginalZip,Latitude,Longitude,Location1,match_addr,match_score,high_conf_match,Number,FiveWords,HP_related,TNumber1,TNumber2,TNumber3,Text1,Text2,Text3,Number_sum,Number_final,Permit Class_y,Units_y,Stories_y,n,n_unit_ratio,n_story_ratio,Permit_Content1,compressor_type_raw,is_VRF_system,is_split_system,is_fan_coil_system
76,6636954-RF,: 6636954-RF : Refrigeration Permit Record Sta...,6717 roosevelt way ne,Lucille on Roosevelt,4,Roosevelt,Seattle,98115,7.0,2019.0,NaN,A,Larry R & Lucille N Pasco,106.0,47.677809,-122.31772,ROOSEVELT/RAVENNA,NaN,NaN,NaN,Mid-Rise,Multifamily,Residential,Refrigeration,NaN,INSTALL PIPING FOR (1) SPLIT SYSTEM AND 6 VRF ...,NaN,NaN,G B SYSTEMS INC,12/27/2017,NaN,6/27/2019,1/10/2019,NaN,2019.0,NaN,NaN,6717 ROOSEVELT WAY NE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://services.seattle.gov/portal/customize/...,NaN,NaN,6717 roosevelt way ne,Trade,12/27/2017,Completed,SEATTLE,WA,98115.0,47.677865,-122.317733,"(47.6778653, -122.31773258)",6717 roosevelt way ne,100,True,1,SPLIT SYSTEM AND 6 VRF SYSTEMS,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1,Multifamily,106.0,7.0,1.0,0.009434,0.142857,: 6636954-RF : Refrigeration Permit Record Sta...,000-005HP,True,True,False
90,6613740-RF,: 6613740-RF : Refrigeration Permit Record Sta...,3501 rainier ave s,Estelle Apartments,3,Columbia City,Seattle,98144,6.0,2018.0,NaN,B,Downtown Emergency Service Center,91.0,47.571578,-122.29344,CLAREMONT/RAINIER VISTA,NaN,NaN,4.0,Mid-Rise,Commercial,Non-Residential,Refrigeration,NaN,REFRIGERATION PIPING FOR (1) 22-TON VRF SYSTEM,NaN,NaN,UNITED SYSTEMS MECHANICAL LLC,8/16/2017,NaN,2/16/2019,1/26/2018,NaN,2018.0,NaN,NaN,3501 RAINIER AVE S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://services.seattle.gov/portal/customize/...,NaN,NaN,3501 rainier ave s,Trade,8/16/2017,Completed,SEATTLE,WA,98144.0,47.571615,-122.293478,"(47.57161504, -122.29347772)",3501 rainier ave s,100,True,1,22-TON VRF SYSTEM,0,1.0,NaN,NaN,22-TON VRF SYSTEM,NaN,NaN,1.0,1.0,Commercial,91.0,6.0,1.0,0.010989,0.166667,: 6613740-RF : Refrigeration Permit Record Sta...,006-025HP,True,False,False
98,6486476-RF,: 6486476-RF : Refrigeration Permit Record Sta...,1900 1st ave,Sequel,4,Pike Place Market,Seattle,98101,11.0,2016.0,NaN,A,1: Peter R Odgaard | 2: Touchstone Corporation,93.0,47.610682,-122.34159,BELLTOWN,NaN,NaN,175.0,Mid-Rise,Commercial,Non-Residential,Refrigeration,NaN,REFER PIPING TO VRF SYSTEMS,NaN,NaN,Holaday Parks Inc,7/27/2015,NaN,1/27/2017,3/10/2016,NaN,2016.0,NaN,NaN,1900 1ST AVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://services.seattle.gov/portal/customize/...,NaN,NaN,1900 1st ave,Trade,7/27/2015,Completed,SEATTLE,WA,98101.0,47.610706,-122.341376,"(47.61070574, -122.34137595)",1900 1st ave,100,True,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Commercial,93.0,11.0,6.0,0.064516,0.545455,: 6486476-RF : Refrigeration Permit Record Sta...,"006-025HP, 000-005HP",True,False,False
129,6906884-RF,: 6906884-RF : Refrigeration Permit Record Sta...,2301 e union st,Midtown Square,4,Mann,Seattle,98122,7.0,2022.0,NaN,A,NaN,432.0,47.612794,-122.30189,CENTRAL AREA/SQUIRE PARK,NaN,NaN,NaN,Mid-Rise,Commercial,Non-Residential,Refrigeration,NaN,PROVIDE AND INSTALL (2) INDOOR F

,Permit Number,Permit_Content,addr_clean_x,BuildingName,StarRating,Submarket,City,ZIPCode,Stories,YearBuilt,YearRenov,Class,TrueOwner,Units,latitude,longitude,Neighborhood,LEED,GreenRating,ParkingSpaces,Style,Permit Class,Permit Class Mapped,Perm,Permit Type Mapped,Description,Appliance Type,Est Project Cost,Contractor Name,Applied Date,Issued Date,Expires Date,Completed Date,Month Applied For,Year Completed,Current Status,PIN,Address,Property Name,Detailed Existing Land Use,Planning Existing Land Use Code,General Existing Land Use,Ownership Type,Number of Buildings,Number of Floors,20K,Existing Building Square Feet,Number of Units,Year Built Group,Year Built,Neighborhoods,Link,Issued,Completed,addr_clean_y,PermitTypeMapped,IssuedDate,StatusCurrent,OriginalCity,OriginalState,OriginalZip,Latitude,Longitude,Location1,match_addr,match_score,high_conf_match,Number,FiveWords,HP_related,TNumber1,TNumber2,TNumber3,Text1,Text2,Text3,Number_sum,Number_final,Permit Class_y,Units_y,Stories_y,n,n_unit_ratio,n_story_ratio,Permit_Content1,compressor_type_raw,is_VRF_system,is_split_system,is_fan_coil_system
0,6360063-RF,: 6360063-RF : Refrigeration Permit Record Sta...,2428 nw market st,AMLI Mark24,4,Ballard,Seattle,98107,6.0,2014.0,NaN,A,AMLI Management Company,304.0,47.668879,-122.38886,BALLARD SOUTH,1.0,platinum,120.0,Mid-Rise,Commercial,Non-Residential,Refrigeration,NaN,REFRIGERATION PIPING FOR 8 SPLIT SYSTEM HEAT P...,NaN,NaN,MERIT MECHANICAL INC,4/5/2013,NaN,10/5/2014,8/13/2014,NaN,2014.0,NaN,NaN,2428 NW MARKET ST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://services.seattle.gov/portal/customize/...,NaN,NaN,2428 nw market st,Trade,4/5/2013,Completed,SEATTLE,WA,98107.0,47.668952,-122.388821,"(47.66895245, -122.38882126)",2428 nw market st,100,True,8,SPLIT SYSTEM HEAT PUMPS,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,8,Commercial,304.0,6.0,1.0,0.003289,0.166667,: 6360063-RF : Refrigeration Permit Record Sta...,000-005HP,False,True,False
2,6091968-RF,: 6091968-RF : Refrigeration Permit Record Sta...,912 dexter ave n,Neptune,4,South Lake Union,Seattle,98109,6.0,2006.0,NaN,A,1: Domain Capital Group | 2: PFA Ejendomme A/S...,234.0,47.627541,-122.34208,QUEEN ANNE,NaN,NaN,714.0,Mid-Rise,Commercial,Non-Residential,Refrigeration,NaN,INSTALL 19 SPLIT SYS REF. PIPING,NaN,NaN,PRO TECH REFRIGERATION LLC,3/20/2006,NaN,9/20/2007,8/2/2006,NaN,2006.0,NaN,NaN,912 DEXTER AVE N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://services.seattle.gov/portal/customize/...,NaN,NaN,912 dexter ave n,Trade,3/20/2006,Completed,SEATTLE,WA,98109.0,47.627610,-122.342118,"(47.62761049, -122.34211852)",912 dexter ave n,100,True,19,SPLIT SYS REF. PIPING,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,19,Commercial,234.0,6.0,1.0,0.004274,0.166667,: 6091968-RF : Refrigeration Permit Record Sta...,000-005HP,False,True,False
7,6482349-RF,: 6482349-RF : Refrigeration Permit Record Sta...,1850 n 34th st,AMLI Wallingford,4,Wallingford,Seattle,98103,4.0,2016.0,NaN,A,AMLI Management Company,231.0,47.648469,-122.33553,WALLINGFORD,NaN,NaN,NaN,Mid-Rise,Commercial,Non-Residential,Refrigeration,NaN,INSTALL [9] SPLIT SYSTEMS. 3/7/16 JCB; ADDRESS...,NaN,NaN,"EMERALD AIRE, INC.",7/1/2015,NaN,1/1/2017,12/5/2016,NaN,2016.0,NaN,NaN,1850 N 34TH ST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://services.seattle.gov/portal/customize/...,NaN,NaN,1850 n 34th st,Trade,7/1/2015,Completed,SEATTLE,WA,98103.0,47.648273,-122.335753,"(47.64827306, -122.33575265)",1850 n 34th st,100,True,16,JCB; ADDRESS CHANGED FROM 3400 WALLINGFORD AVE...,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,16,Commercial,231.0,4.0,1.0,0.004329,0.250000,: 6482349-RF : Refrigeration Permit Record Sta...,000-005HP,False,True,False
8,6392865-RF,: 6392865-RF : Refrigeration Permit Record Sta...,4516 meridian ave n,Wallingford Studios,3,Wallingford,Seattle,98103,5.0,2013.0,NaN,B,Footprint Investments LLC,58.0,47.662060,-122.33340,WALLINGFORD,NaN,NaN,12.0,Mid-Rise,Commercial,Non-Residential,Refrigeration,NaN,INSTALL (1) DUCTLESS SPLIT SYSTEM,

In [26]:
df_final2= pd.read_excel('02ProcessedData/apt_trade_HP_permit_details_1028.xlsx')

df_apt_unique_address = pd.read_excel('02ProcessedData/data_apt_unique_address.xlsx')
df_permit_parsed= pd.read_excel('02ProcessedData/parsed_permit_details_new_method.xlsx')
##join df_final2 with df_permit_parsed on permit_id to get the final dataframe with parsed permit details

df_final3 = df_final2.merge(df_permit_parsed, left_on='Permit Number', right_on='permit_id', how='left', suffixes=('', '_y'))
df_final3.info()
df_final3.to_excel('02ProcessedData/final_apt_trade_with_parsed_permit_details1020_2025.xlsx', index=False)

## dfinal3 summary by addr_clean_y and sum of tonnage_total and compressors_final
# df_final3_summary = df_final3.groupby('addr_clean_y').agg({'tonnage_total': 'sum', 'compressors_final': 'sum', 'Units_y':'mean', 'Existing Building Square Feet': 'mean', 'Stories_y': 'mean'}).reset_index()
# df_final3_summary['tonnage_cover_ratio'] = df_final3_summary['tonnage_total']* 700 / (df_final3_summary['Existing Building Square Feet'] )
# df_final3_summary['compressors_per_units'] = df_final3_summary['compressors_final'] / df_final3_summary['Units_y']

####### for df_final3, do a running total of tonnage_total and compressors_final and number_final by addr_clean_y and Year Completed,sorted by Year Completed
df_final3_summary = df_final3
df_final3_summary = df_final3_summary.sort_values(by=['addr_clean_y', 'Year Completed'])
df_final3_summary['tonnage_total_cumsum'] = df_final3_summary.groupby('addr_clean_y')['tonnage_total'].cumsum()
df_final3_summary['compressors_final_cumsum'] = df_final3_summary.groupby('addr_clean_y')['compressors_final'].cumsum()
# df_final3_summary['number_final_cumsum'] = df_final3_summary.groupby('addr_clean_y')['number_final'].cumsum()

df_final3_summary['tonnage_cover_ratio'] = df_final3_summary['tonnage_total_cumsum']* 700 / (df_final3_summary['Existing Building Square Feet'] )
df_final3_summary['compressors_per_units'] = df_final3_summary['compressors_final_cumsum'] / df_final3_summary['Units_y']
# df_final3_summary['number_final_unit_ratio'] = df_final3_summary['number_final_cumsum'] / df_final3_summary['Units_y']

## merge df_final3_summary with df_apt_unique_address on addr_clean_y and addr_clean
df_final3_summary = df_final3_summary.merge(df_apt_unique_address, left_on='addr_clean_y', right_on='addr_clean', how='left', suffixes=('', '_y'))
# df_final3_summary.to_excel('02ProcessedData/final_apt_trade_with_parsed_permit_details1020_2025_summary.xlsx', index=False)
df_final3_summary.describe().T






##merge df_final3_summary with df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 96 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Permit Number                    934 non-null    object 
 1   Permit_Content                   934 non-null    object 
 2   addr_clean_x                     934 non-null    object 
 3   BuildingName                     923 non-null    object 
 4   StarRating                       934 non-null    int64  
 5   Submarket                        934 non-null    object 
 6   City                             934 non-null    object 
 7   ZIPCode                          934 non-null    int64  
 8   Stories                          933 non-null    float64
 9   YearBuilt                        931 non-null    float64
 10  YearRenov                        38 non-null     float64
 11  Class                            934 non-null    object 
 12  TrueOwner             

,count,mean,std,min,25%,50%,75%,max
StarRating,935.0,3.835294e+00,5.180214e-01,2.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,5.000000e+00
ZIPCode,935.0,9.811376e+04,1.148455e+01,9.810100e+04,9.810400e+04,9.810900e+04,9.812200e+04,9.819900e+04
Stories,934.0,7.718415e+00,6.562009e+00,1.000000e+00,5.000000e+00,6.000000e+00,7.000000e+00,4.400000e+01
YearBuilt,932.0,2.007722e+03,2.328267e+01,1.900000e+03,2.009000e+03,2.015000e+03,2.019000e+03,2.023000e+03
YearRenov,38.0,2.010500e+03,9.802785e+00,1.978000e+03,2.006000e+03,2.015000e+03,2.016000e+03,2.019000e+03
Units,935.0,1.508770e+02,1.199816e+02,1.500000e+01,6.000000e+01,1.080000e+02,2.295000e+02,7.610000e+02
latitude,935.0,4.762595e+01,4.041767e-02,4.752012e+01,4.761004e+01,4.761877e+01,4.765654e+01,4.773255e+01
longitude,935.0,-1.223313e+02,2.606837e-02,-1.223889e+02,-1.223469e+02,-1.223276e+02,-1.223152e+02,-1.222617e+02
LEED,141.0,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
ParkingSpaces,491.0,1.426500e+02,1.323899e+02,0.000000e+00,4.000000e+01,1.100000e+02,2.000000e+02,9.160000e+02


In [ ]:
## get the project value from 

# df.columns
# df_sum1 = df.groupby('addr_clean_x').agg({'Est Project Cost': 'sum', 'Units': 'mean'}).reset_index()
# df_sum1['cost_unit_ratio'] = abs(df_sum1['Est Project Cost']/df_sum1['Units'])
# df_sum1.head()


Index(['Number', 'Quarter', 'Status', 'AskingRentPerUnit', 'AskingRentPerSF', 'AskingRentGrowthYr',
       'EffectiveRentPerUnit', 'EffectiveRentPerSF', 'EffectiveRentGrowthYr', 'VacancyUnits', 'VacancyTotal',
       'Address', 'BuildingName', 'StarRating', 'ES', 'Esyearscertified', 'Eslast', 'LEED', 'GreenRating', 'LEEDtype',
       'Points', 'version', 'RBAGLA', 'Submarket', 'City', 'ZIPCode', 'Stories', 'YearBuilt', 'YearRenov', 'Class',
       'TrueOwner', 'Units', 'Style', 'AvgConcessions', 'StudiosUnits', 'Onebedunits', 'Twobedunits', 'Threebedunits',
       'Fourbedunits', 'RentType', 'AffordableType', 'ParkingSpacesUnit', 'ParkingSpaces', 'MarketSegment', 'latitude',
       'longitude', 'Neighborhood', 'addr_clean', 'has heat pump', 'addr_clean_x', 'coverage_classification_final',
       'Completed Date', 'HP_related', 'Year Completed', 'Quarter_clean', 'Quarter_period', 'Quarter_end',
       'Completed_ts', 'is_before', 'is_after', 'is_same_or_after', 'HP_installed'],
      dt

KeyError: "Column(s) ['Est Project Cost'] do not exist"

## Fast Rule of Thumb for Deciding
1. mini-split/ducted unitary: Number of Compressors == Number of Units
2. Central VRF/VRV:
    - Typical diversified peak per unit (tons):
    - Low-rise: 1.4–1.8 • Mid-rise: 1.1–1.5 • High-rise: 0.9–1.3
    - Common outdoor module “size”: 12–20 tons (each module usually has ~1 inverter compressor; treat 1 module ≈ 1 compressor for counting).
    - Low-rise: ~1 compressor per 9–11 units
    - Mid-rise: ~1 per 11–13 units
    - High-rise: ~1 per 13–16 units
3. Central air-to-water heat pumps + fan-coils
    Heuristic using 60-ton modules:
        • Low-rise (~1.6 t/u): 1 compressor per ~35–40 units
        • Mid-rise (~1.3 t/u): 1 per ~45–55 units
        • High-rise (~1.1 t/u): 1 per ~55–65 units
4. WSHP (water-source heat pump) in each unit + central loop plant
    In-unit ==  compressors (each suite has its own WSHP).

In [30]:
import numpy as np
## Redo classification based on tonnage_cover_ratio and compressors_per_units
## if tonnage_cover_ratio >= 0.8 --> 'for whole building'
## if (Style is 'Mid-Rise' and tonnage_cover_ratio <= 0.2) | (Style is 'Low-Rise' and tonnage_cover_ratio <= 0.15) | (Style is 'Hi-Rise' and tonnage_cover_ratio <= 0.25) --> 'for common areas only'
conditions = [
    df_final3_summary['tonnage_cover_ratio'] >= 0.8,
    ((df_final3_summary['Style'] == 'Mid-Rise') & (df_final3_summary['tonnage_cover_ratio'] <= 0.2 ) & (df_final3_summary['tonnage_cover_ratio'] > 0)) |
    ((df_final3_summary['Style'] == 'Low-Rise') & (df_final3_summary['tonnage_cover_ratio'] <= 0.15) & (df_final3_summary['tonnage_cover_ratio'] > 0)) |
    ((df_final3_summary['Style'] == 'Hi-Rise') & (df_final3_summary['tonnage_cover_ratio'] <= 0.25) & (df_final3_summary['tonnage_cover_ratio'] > 0)),
     (df_final3_summary['tonnage_cover_ratio'] > 0),
     (df_final3_summary['compressors_per_units'] >= 0.75),
     (df_final3_summary['compressors_per_units'] < 0.02),
    (df_final3_summary['compressors_per_units'] <= 0.75) &      (df_final3_summary['compressors_per_units'] >= 0.4)
]


choices = ['Whole Building', 'Common Areas Only', 'Partial Coverage','Whole Building', 'Common Areas Only', 'Partial Coverage']
df_final3_summary['coverage_classification_tonnage'] = np.select(conditions, choices, default='Uncertain')

df_final3_summary['coverage_classification_tonnage'].value_counts()



coverage_classification_tonnage
Uncertain            660
Common Areas Only    212
Whole Building        35
Partial Coverage      28
Name: count, dtype: int64

In [33]:

## join df_final2 with df_final3_summary on addr_clean_y to get the coverage_classification_tonnage back to df_final3

## find columns unique in df_final3_summary to merge back to df_final2
unique_cols = df_final3_summary.columns.difference(df_final2.columns)

## find columns that are shared between df_final2 and df_final3_summary
shared_cols = df_final3_summary.columns.intersection(df_final2.columns)
# unique_cols
# shared_cols
keep_columns = ['permit_id', 'tonnage_cover_ratio',
       'tonnage_total', 'tonnage_total_cumsum', 'compressors_per_units',
       'compressors_final', 'compressors_final_cumsum', 'coverage_classification_tonnage']
df_final4 = df_final2.merge(df_final3_summary[keep_columns], left_on='Permit Number', right_on='permit_id', how='left')
# df_final4 = df_final4.merge(df_sum1[['addr_clean_x', 'cost_unit_ratio']], left_on='addr_clean_y', right_on='addr_clean_x', how='left')
df_final4.info()

# df_final4.to_excel('02ProcessedData/final_apt_trade_with_parsed_permit_details1020_2025_with_classification.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 935 entries, 0 to 934
Data columns (total 97 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Permit Number                    935 non-null    object 
 1   Permit_Content                   935 non-null    object 
 2   addr_clean_x                     935 non-null    object 
 3   BuildingName                     924 non-null    object 
 4   StarRating                       935 non-null    int64  
 5   Submarket                        935 non-null    object 
 6   City                             935 non-null    object 
 7   ZIPCode                          935 non-null    int64  
 8   Stories                          934 non-null    float64
 9   YearBuilt                        932 non-null    float64
 10  YearRenov                        38 non-null     float64
 11  Class                            935 non-null    object 
 12  TrueOwner             

In [34]:

###- if 'Detailed Existing Land Use' containts 'mixed use|condo' & - if 'Description' contains 'lounge|single zone|lobby' --> df_final['common_area_only'] = 1

import numpy as np

df_final4['common_area_only'] = np.nan
df_final4['Number_final'] = pd.to_numeric(df_final4['Number_final'], errors='coerce')
df_final4['Units_y'] = pd.to_numeric(df_final4['Units_y'], errors='coerce')

## sort df_final4 by 'addr_clean_y' and 'Year Completed' and then do a running total of 'Number_final' by 'addr_clean_y'
df_final4 = df_final4.sort_values(by=['addr_clean_y', 'Year Completed'])
df_final4['Number_final_cumsum'] = df_final4.groupby('addr_clean_y')['Number_final'].cumsum()
df_final4['n_unit_ratio'] = df_final4['Number_final_cumsum'] / df_final4['Units_y']
# Ensure numeric columns are clean

##Note that when using np.select(conditions, choices), the first condition that evaluates to True for a given row is the one that gets applied. Later rules are ignored for that row once a match is found.

# Define conditions
conditions = [

    # Rule 2: Shared housing with very low unit ratio --> for common areas
    df_final4['Permit Class'].str.contains(
        'Commercial|Multifamily', case=False, na=False) & df_final4['is_split_system']==True &
    (df_final4['n_unit_ratio'] <= (1 / df_final4['Stories'])),

    ## Rule 3: any cases that n_unit_ratio >=1 & not negative Number of Buildings --> for units
    (df_final4['n_unit_ratio'] >= 0.8)  ,
    (df_final4['n_unit_ratio'] <= 0.8) & (df_final4['n_unit_ratio'] >= 0.3) ,

        # Rule 1: Common space keywords in shared housing --> for common areas
   df_final4['Permit Class'].str.contains(
        'Commercial|Multifamily', case=False, na=False
    ) & df_final4['Description'].str.contains(
        'lounge|single zone|lobby|retail', case=False, na=False
    ),


]


## choices = ['Whole Building', 'Common Areas Only', 'Partial Coverage']


# Corresponding values
choices = ['Common Areas Only', 'Whole Building', 'Partial Coverage' , 'Common Areas Only']

# Apply CASE WHEN logic
df_final4['coverage_classification_units'] = np.select(conditions, choices, default= 'Uncertain')
df_final4['coverage_classification_units'].value_counts()

df_final4[['coverage_classification_tonnage','coverage_classification_units']].value_counts()

df_final4['coverage_classification_final'] = np.where(df_final4['coverage_classification_tonnage'] == 'Uncertain', df_final4['coverage_classification_units'], df_final4['coverage_classification_tonnage'])

df_final4['coverage_classification_final'] = np.where(df_final4['Description'].str.contains('each unit', case=False, na=False), "Whole Building", df_final4['coverage_classification_final'])

df_final4['coverage_classification_final'].value_counts()

df_final4[['coverage_classification_tonnage','coverage_classification_units', 'coverage_classification_final']].head(5)
# df_final4.to_excel('03ExportedData/final_apt_trade_with_parsed_permit_details1020_2025_with_classification_v2.xlsx', index=False)



,coverage_classification_tonnage,coverage_classification_units,coverage_classification_final
369,Common Areas Only,Partial Coverage,Common Areas Only
371,Uncertain,Common Areas Only,Common Areas Only
370,Common Areas Only,Uncertain,Common Areas Only
502,Common Areas Only,Uncertain,Common Areas Only
497,Common Areas Only,Common Areas Only,Common Areas Only


In [38]:
df_final4[df_final4['BuildingName'] == 'Northaven Apartments'].head()
df_final4['HP_related'].value_counts()
df_final4.describe().T

,count,mean,std,min,25%,50%,75%,max
StarRating,935.0,3.835294e+00,5.180214e-01,2.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,5.000000e+00
ZIPCode,935.0,9.811376e+04,1.148455e+01,9.810100e+04,9.810400e+04,9.810900e+04,9.812200e+04,9.819900e+04
Stories,934.0,7.718415e+00,6.562009e+00,1.000000e+00,5.000000e+00,6.000000e+00,7.000000e+00,4.400000e+01
YearBuilt,932.0,2.007722e+03,2.328267e+01,1.900000e+03,2.009000e+03,2.015000e+03,2.019000e+03,2.023000e+03
YearRenov,38.0,2.010500e+03,9.802785e+00,1.978000e+03,2.006000e+03,2.015000e+03,2.016000e+03,2.019000e+03
Units,935.0,1.508770e+02,1.199816e+02,1.500000e+01,6.000000e+01,1.080000e+02,2.295000e+02,7.610000e+02
latitude,935.0,4.762595e+01,4.041767e-02,4.752012e+01,4.761004e+01,4.761877e+01,4.765654e+01,4.773255e+01
longitude,935.0,-1.223313e+02,2.606837e-02,-1.223889e+02,-1.223469e+02,-1.223276e+02,-1.223152e+02,-1.222617e+02
LEED,141.0,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
ParkingSpaces,491.0,1.426500e+02,1.323899e+02,0.000000e+00,4.000000e+01,1.100000e+02,2.000000e+02,9.160000e+02


coverage_classification_final
Common Areas Only    573
Uncertain            259
Whole Building        40
Partial Coverage      28
Name: count, dtype: int64

In [49]:
df_final_final  = df_final4.copy()
# df_final_final['Year Completed'] = np.where(df_final_final['Year Completed'].isnull(), pd.to_datetime(df_final_final['Expires Date']).dt.year, df_final_final['Year Completed'])
# df_final_final['Year Completed'] =df_final_final['Year Completed_x'] 
df_final_final['Year Completed'] = np.where(
    df_final_final['Year Completed'].isnull(),
    pd.to_datetime(df_final_final['Expires Date'], errors='coerce').dt.year,
    df_final_final['Year Completed']
)

## keep some columns only
columns_to_keep = ['addr_clean_x','coverage_classification_final', 'Completed Date', 'HP_related', 'Year Completed']
other_cols = df_apt_unique_address.columns.tolist()
df_final_final = df_final_final[columns_to_keep ].drop_duplicates().reset_index(drop=True)

## make column to upper case
df_final_final['addr_clean'] = df_final_final['addr_clean_x'].str.upper()
df_final_final['HP_covered_residential_units'] = np.where(df_final_final['coverage_classification_final'].isin(['Whole Building','Partial Coverage']), 1, 0)
df_final_final['coverage_classification_final'] =df_final_final['coverage_classification_final'].fillna('No Permit Found')

## fill na of df_final_final['coverage_classification_ final'] with 'No Heat Pump Installed'
df_final_final['coverage_classification_final'] = df_final_final['coverage_classification_final'].fillna('No Heat Pump Installed')
df_final_final = df_final_final.drop_duplicates(subset=['addr_clean',]).reset_index(drop=True)
df_final_final.to_excel('03ExportedData/final_apt_trade_address_classification_1029_2025.xlsx', index=False)



In [ ]:

df_ht = df_apt_unique_address.merge(df_final_final[['addr_clean_x', 'coverage_classification_final','HP_related', 'Year Completed']], left_on='addr_clean', right_on='addr_clean_x', how='left')
df_ht['coverage_classification_final'].value_counts()
## fill na in coverage_classification_final as 'No Permit Found'
## fill na in HP_related as 0
df_ht['HP_related'] = df_ht['HP_related'].fillna(0)

# df_ht['HP_covered_residential_units'] = np.where(df_ht['coverage_classification_final'].isin(['Whole Building','Partial Coverage']), 1, 0)
df_ht = df_ht.drop_duplicates(subset=['addr_clean']).reset_index(drop=True)
df_ht.info()
df_ht.to_excel('03ExportedData/final_apt_trade_address_classification_1024_2025.xlsx', index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   addr_clean                     943 non-null    object 
 1   BuildingName                   917 non-null    object 
 2   StarRating                     943 non-null    int64  
 3   Submarket                      943 non-null    object 
 4   City                           943 non-null    object 
 5   ZIPCode                        943 non-null    int64  
 6   Stories                        941 non-null    float64
 7   YearBuilt                      931 non-null    float64
 8   YearRenov                      64 non-null     float64
 9   Class                          943 non-null    object 
 10  TrueOwner                      918 non-null    object 
 11  Units                          942 non-null    float64
 12  latitude                       943 non-null    flo

## Do a fuzzy join on the addr_clean


In [41]:
import pandas as pd

data_apt_time = pd.read_excel('01Data/Seattle_HP_Aptmt Units_Timeseries.xlsx')


In [50]:
df_final_final = pd.read_excel('03ExportedData/final_apt_trade_address_classification_1029_2025.xlsx')


In [53]:
df_final_final['coverage_classification_final'] = df_final_final['coverage_classification_final'].fillna('No Heat Pump Installed')
df_final_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   addr_clean_x                   493 non-null    object 
 1   coverage_classification_final  493 non-null    object 
 2   Completed Date                 471 non-null    object 
 3   HP_related                     493 non-null    int64  
 4   Year Completed                 473 non-null    float64
 5   addr_clean                     493 non-null    object 
 6   HP_covered_residential_units   493 non-null    int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 27.1+ KB


In [54]:

df= data_apt_time
df.info()
df['addr_clean'].head()

df = df.merge(df_final_final, left_on='addr_clean', right_on='addr_clean', how='left')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47905 entries, 0 to 47904
Data columns (total 49 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Number                 47905 non-null  object 
 1   Quarter                47905 non-null  object 
 2   Status                 47905 non-null  object 
 3   AskingRentPerUnit      42015 non-null  float64
 4   AskingRentPerSF        41455 non-null  float64
 5   AskingRentGrowthYr     41157 non-null  float64
 6   EffectiveRentPerUnit   42015 non-null  float64
 7   EffectiveRentPerSF     41455 non-null  float64
 8   EffectiveRentGrowthYr  41157 non-null  float64
 9   VacancyUnits           47828 non-null  float64
 10  VacancyTotal           47828 non-null  float64
 11  Address                47905 non-null  object 
 12  BuildingName           46519 non-null  object 
 13  StarRating             47905 non-null  int64  
 14  ES                     2450 non-null   float64
 15  Es

In [55]:
df.info()
df.columns


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47905 entries, 0 to 47904
Data columns (total 55 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Number                         47905 non-null  object 
 1   Quarter                        47905 non-null  object 
 2   Status                         47905 non-null  object 
 3   AskingRentPerUnit              42015 non-null  float64
 4   AskingRentPerSF                41455 non-null  float64
 5   AskingRentGrowthYr             41157 non-null  float64
 6   EffectiveRentPerUnit           42015 non-null  float64
 7   EffectiveRentPerSF             41455 non-null  float64
 8   EffectiveRentGrowthYr          41157 non-null  float64
 9   VacancyUnits                   47828 non-null  float64
 10  VacancyTotal                   47828 non-null  float64
 11  Address                        47905 non-null  object 
 12  BuildingName                   46519 non-null 

Index(['Number', 'Quarter', 'Status', 'AskingRentPerUnit', 'AskingRentPerSF',
       'AskingRentGrowthYr', 'EffectiveRentPerUnit', 'EffectiveRentPerSF',
       'EffectiveRentGrowthYr', 'VacancyUnits', 'VacancyTotal', 'Address',
       'BuildingName', 'StarRating', 'ES', 'Esyearscertified', 'Eslast',
       'LEED', 'GreenRating', 'LEEDtype', 'Points', 'version', 'RBAGLA',
       'Submarket', 'City', 'ZIPCode', 'Stories', 'YearBuilt', 'YearRenov',
       'Class', 'TrueOwner', 'Units', 'Style', 'AvgConcessions',
       'StudiosUnits', 'Onebedunits', 'Twobedunits', 'Threebedunits',
       'Fourbedunits', 'RentType', 'AffordableType', 'ParkingSpacesUnit',
       'ParkingSpaces', 'MarketSegment', 'latitude', 'longitude',
       'Neighborhood', 'addr_clean', 'has heat pump', 'addr_clean_x',
       'coverage_classification_final', 'Completed Date', 'HP_related',
       'Year Completed', 'HP_covered_residential_units'],
      dtype='object')

Number	Quarter	Status	AskingRentPerUnit	AskingRentPerSF	AskingRentGrowthYr	EffectiveRentPerUnit	EffectiveRentPerSF	EffectiveRentGrowthYr	VacancyUnits	VacancyTotal	Address	BuildingName	StarRating	ES	Esyearscertified	Eslast	LEED	GreenRating	LEEDtype	Points	version	RBAGLA	Submarket	City	ZIPCode	Stories	YearBuilt	YearRenov	Class	TrueOwner	Units	Style	AvgConcessions	StudiosUnits	Onebedunits	Twobedunits	Threebedunits	Fourbedunits	RentType	AffordableType	ParkingSpacesUnit	ParkingSpaces	MarketSegment	latitude	longitude	Neighborhood	addr_clean	has heat pump			


In [56]:
import pandas as pd

# Work on a copy of the original dataset
# Clean and standardize the 'Quarter' column
df['Quarter_clean'] = (
    df['Quarter']
    .str.replace('QTD', '', regex=False)
    .str.strip()
    .str.replace(' ', '', regex=False)
)

# Convert cleaned quarter strings to quarterly periods
df['Quarter_period'] = pd.PeriodIndex(df['Quarter_clean'], freq='Q')

# Convert to timestamp (start of quarter) and then shift to quarter end
df['Quarter_end'] = df['Quarter_period'].dt.to_timestamp() + pd.offsets.QuarterEnd(0)


## check if 'Completed Date' column exists, if not Completed Date is equal to Completed Date_y

df['Completed_ts'] = pd.to_datetime(df['Completed Date'], errors='coerce')

# Compare timestamps
df['is_before'] = df['Completed_ts'] < df['Quarter_end']
df['is_after'] = df['Completed_ts'] > df['Quarter_end']
df['is_same_or_after'] = df['Completed_ts'] >= df['Quarter_end']


In [59]:


### make the judgement based on whether the Completed Date is before or after the Quarter_end
### if Completed_ts is NaT , then set HP_installed = 0
### if HP_related == 0, then set HP_installed = 0
### if Completed_ts is before Quarter_end & HP_related == 1, then set HP_installed = 1
### if Completed_ts is same or after Quarter_end, then set HP_installed = 0
df['HP_installed'] = 0
df.loc[(df['Completed_ts'].notna()) & (df['HP_related'] == 1) & (df['is_before']), 'HP_installed'] = 1
df['HP_installed'].value_counts()
# df[df['HP_installed'] == 1][['Address', 'addr_clean', 'Quarter', 'Completed Date', 'Completed_ts', 'Quarter_end', 'is_before', 'HP_related', 'HP_installed']].head(20)

df[df['HP_installed'] == 1]['addr_clean'].value_counts()
df['HP_covered_residential_units'].value_counts()
df[df['HP_covered_residential_units'] == 1]['addr_clean'].nunique()
df['coverage_classification_final'] = df['coverage_classification_final'].fillna('No Heat Pump Installed')




In [60]:
df.head(10)
df['coverage_classification_final'].value_counts()

coverage_classification_final
No Heat Pump Installed    28698
Common Areas Only         13927
Uncertain                  4586
Whole Building              458
Partial Coverage            236
Name: count, dtype: int64

In [61]:
df.to_excel('03ExportedData/time_series_with_hp.xlsx', index=False)
